In [1]:
import pandas as pd
import numpy as np
import glob
import os
from openpyxl import load_workbook
pd.options.display.float_format = '{:,.2f}'.format

# SNOWFLAKE DATA

In [2]:
sql_prov_1 = pd.read_csv(r'G:\My Drive\3.-Settlement\PROVISIONES\2022\04. ABRIL\PROV 28 ABRIL.csv',
                         dtype={'AMOUNT':float,'CLAVE_AUTORIZACION': object,'BIN':object,'LAST4':object,'ORDER_ID':object,
                                'DEVICE_ID':object})
sql_prov_2 = pd.read_csv(r'G:\My Drive\3.-Settlement\PROVISIONES\2022\04. ABRIL\PROV 29 ABRIL.csv'
                         ,dtype={'AMOUNT':float,'CLAVE_AUTORIZACION': object,'BIN':object,'LAST4':object,'ORDER_ID':object,
                                 'DEVICE_ID':object})
sql_prov_3 = pd.read_csv(r'G:\My Drive\3.-Settlement\PROVISIONES\2022\04. ABRIL\PROV 30 ABRIL.csv',
                         dtype={'AMOUNT':float,'CLAVE_AUTORIZACION': object,'BIN':object,'LAST4':object,'ORDER_ID':object,
                                'DEVICE_ID':object})

In [3]:
sql1 = pd.concat([sql_prov_1,sql_prov_2,sql_prov_3])
sql1.columns = sql1.columns.str.strip().str.lower()
sql1['msi'].fillna(0,inplace=True)
sql1.head()

,merchant_id,transaction_id,order_id,psp,affiliation_type,clave_autorizacion,date_transaction,amount,bin,last4,status_code,msi,clip_reader,report_id,report_date_local,status
0,1f8d4205-90c8-4620-b0ca-44f9a050198d,979b4ff1-5e1a-4656-993a-bcbbe6f94d88,247352693415,PROSA,CARD_PRESENT,NaN,2022-04-28,280.00,407561,4940,-3,0.00,Clip Pro,NaN,NaN,NaN
1,2bb7d4f8-e3d5-4d00-8a48-1330fddbaf29,1a2ad035-dcb1-4b3b-9793-16e27565ff61,562598437233,PROSA,CARD_PRESENT,000047,2022-04-28,137.00,491566,0061,1,0.00,Clip Pro,TPWH4GC2P,2022-04-29,PAID
2,183dfdc9-6976-4721-b4b6-142ec26a3c14,bf134325-4700-466d-be69-1d56d0ff1634,321943055354,PROSA,CARD_PRESENT,446966,2022-04-28,67.00,402766,6789,1,0.00,Clip Plus,LT2V7YBTR,2022-04-29,PAID
3,c90d40c5-1602-465d-811b-7824f0d0beec,5f4f5558-75e5-42e8-bd7a-279031d3fc80,152222294174,PAYWORKS-BANORTE,ECOMMERCE,NaN,2022-04-28,"2,100.00",526777,5482,-3,0.00,Clip Classic,NaN,NaN,NaN
4,529367d5-31de-4c72-8e3c-b0bd75299afe,bc455f37-2a77-4245-94d2-44411ece89d3,892351086541,PROSA,CARD_PRESENT,000468,2022-04-28,90.00,557908,3485,1,0.00,Clip Plus,69Y3SYCRP,2022-04-29,PAID


In [4]:
sql1.shape

(1300619, 16)

In [5]:
all_provisiones = sql1[(sql1['status_code'] == 1)]
all_provisiones.head()

,merchant_id,transaction_id,order_id,psp,affiliation_type,clave_autorizacion,date_transaction,amount,bin,last4,status_code,msi,clip_reader,report_id,report_date_local,status
1,2bb7d4f8-e3d5-4d00-8a48-1330fddbaf29,1a2ad035-dcb1-4b3b-9793-16e27565ff61,562598437233,PROSA,CARD_PRESENT,000047,2022-04-28,137.00,491566,0061,1,0.00,Clip Pro,TPWH4GC2P,2022-04-29,PAID
2,183dfdc9-6976-4721-b4b6-142ec26a3c14,bf134325-4700-466d-be69-1d56d0ff1634,321943055354,PROSA,CARD_PRESENT,446966,2022-04-28,67.00,402766,6789,1,0.00,Clip Plus,LT2V7YBTR,2022-04-29,PAID
4,529367d5-31de-4c72-8e3c-b0bd75299afe,bc455f37-2a77-4245-94d2-44411ece89d3,892351086541,PROSA,CARD_PRESENT,000468,2022-04-28,90.00,557908,3485,1,0.00,Clip Plus,69Y3SYCRP,2022-04-29,PAID
5,f63917ba-4561-4533-88e7-5ab9f3c3248e,e92d391d-2172-4dc5-a96f-e611bedab6ac,537981910143,PROSA,CARD_PRESENT,006024,2022-04-28,73.00,514065,4821,1,0.00,Clip Total,TGCCGW4YB,2022-04-29,PAID
6,406e0bef-bb42-452e-9056-5522662d1991,67c1271e-b4c9-4da8-af85-bbcd8bbe31b6,830007589726,PROSA,CARD_PRESENT,994592,2022-04-28,200.00,542878,1133,1,0.00,Clip Plus 2,NHH4KMBG8,2022-04-29,PAID


In [6]:
all_provisiones.shape

(1210938, 16)

In [7]:
all_provisiones['psp'].unique()

array(['PROSA', 'AMEX', 'PAYWORKS-BANORTE', 'MIT-3DS'], dtype=object)

In [8]:
all_provisiones.dtypes

merchant_id            object
transaction_id         object
order_id               object
psp                    object
affiliation_type       object
clave_autorizacion     object
date_transaction       object
amount                float64
bin                    object
last4                  object
status_code             int64
msi                   float64
clip_reader            object
report_id              object
report_date_local      object
status                 object
dtype: object

# PROVISIONES BANORTE

In [9]:
provision_bnte = all_provisiones[(all_provisiones['psp'] == 'PROSA')]
provision_bnte['CONCAT'] = (provision_bnte['order_id'].astype(str).replace(' ', '',regex=True)+'-'
                            +provision_bnte['amount'].apply(np.floor).astype(str).replace(' ', '',regex=True)+'-'
                            +provision_bnte['last4'].astype(str).replace(' ', '',regex=True))
provision_bnte.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_13792\763977856.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  provision_bnte['CONCAT'] = (provision_bnte['order_id'].astype(str).replace(' ', '',regex=True)+'-'


,merchant_id,transaction_id,order_id,psp,affiliation_type,clave_autorizacion,date_transaction,amount,bin,last4,status_code,msi,clip_reader,report_id,report_date_local,status,CONCAT
1,2bb7d4f8-e3d5-4d00-8a48-1330fddbaf29,1a2ad035-dcb1-4b3b-9793-16e27565ff61,562598437233,PROSA,CARD_PRESENT,000047,2022-04-28,137.00,491566,0061,1,0.00,Clip Pro,TPWH4GC2P,2022-04-29,PAID,562598437233-137.0-0061
2,183dfdc9-6976-4721-b4b6-142ec26a3c14,bf134325-4700-466d-be69-1d56d0ff1634,321943055354,PROSA,CARD_PRESENT,446966,2022-04-28,67.00,402766,6789,1,0.00,Clip Plus,LT2V7YBTR,2022-04-29,PAID,321943055354-67.0-6789
4,529367d5-31de-4c72-8e3c-b0bd75299afe,bc455f37-2a77-4245-94d2-44411ece89d3,892351086541,PROSA,CARD_PRESENT,000468,2022-04-28,90.00,557908,3485,1,0.00,Clip Plus,69Y3SYCRP,2022-04-29,PAID,892351086541-90.0-3485
5,f63917ba-4561-4533-88e7-5ab9f3c3248e,e92d391d-2172-4dc5-a96f-e611bedab6ac,537981910143,PROSA,CARD_PRESENT,006024,2022-04-28,73.00,514065,4821,1,0.00,Clip Total,TGCCGW4YB,2022-04-29,PAID,537981910143-73.0-4821
6,406e0bef-bb42-452e-9056-5522662d1991,67c1271e-b4c9-4da8-af85-bbcd8bbe31b6,830007589726,PROSA,CARD_PRESENT,994592,2022-04-28,200.00,542878,1133,1,0.00,Clip Plus 2,NHH4KMBG8,2022-04-29,PAID,830007589726-200.0-1133


In [10]:
provision_bnte.shape

(1184975, 17)

### RUTAS DE ACCESO SFTP

In [11]:
name_file=['column_type','affiliation_sftp','merchant_reference','card','settled_amount','commisions','transaction_date',
           'gross_amount','currency','transaction_type','tax','installments_tax','iva_cashback_paid','iva_cashback_charged',
          'cashback_fees_paid','cashback_fees_charged','installments_fees','cashback_amount','number_of_payments',
           'initial_deferment','plan_type','auth_code','unknown.1','eci','interbank_reference','empty.1','unknown.2','empty.2',
          'payworks_reference','card_type','transaction_source','prosa_reference','acquirer_commision','acquirer_tax',
          'unknown.3','unknown.4','unknown.5']
dtypes_file={'interbank_reference':object,'affiliation_sftp':object,'card':object,'number_of_payments':object,
             'initial_deferment':object,'plan_type':object,'auth_code':object,'payworks_reference':object}

In [12]:
use_cols = ['affiliation_sftp','transaction_type','transaction_date','gross_amount','auth_code','interbank_reference',
            'payworks_reference','card','commisions','tax','installments_fees','installments_tax','acquirer_commision',
            'acquirer_tax']

In [13]:
sftp_prov_files = glob.glob(r'G:\My Drive\3.-Settlement\PROVISIONES\2022\04. ABRIL\SFTP\BNTE\PAYCLIP_Settlement*.csv',
                            recursive=False)
sftp_prov_files

['G:\\My Drive\\3.-Settlement\\PROVISIONES\\2022\\04. ABRIL\\SFTP\\BNTE\\PAYCLIP_Settlement_29042022_V1.csv',
 'G:\\My Drive\\3.-Settlement\\PROVISIONES\\2022\\04. ABRIL\\SFTP\\BNTE\\PAYCLIP_Settlement_30042022_V1.csv',
 'G:\\My Drive\\3.-Settlement\\PROVISIONES\\2022\\04. ABRIL\\SFTP\\BNTE\\PAYCLIP_Settlement_28042022_V1.csv',
 'G:\\My Drive\\3.-Settlement\\PROVISIONES\\2022\\04. ABRIL\\SFTP\\BNTE\\PAYCLIP_Settlement_27042022_V1.csv',
 'G:\\My Drive\\3.-Settlement\\PROVISIONES\\2022\\04. ABRIL\\SFTP\\BNTE\\PAYCLIP_Settlement_03052022_V1.csv',
 'G:\\My Drive\\3.-Settlement\\PROVISIONES\\2022\\04. ABRIL\\SFTP\\BNTE\\PAYCLIP_Settlement_04052022_V1.csv']

In [14]:
sftp = pd.concat([pd.read_csv(f,names=name_file,dtype=dtypes_file,usecols=use_cols)
                         .assign(file_name=os.path.basename(f)) for f in sftp_prov_files])
sftp.rename(columns ={'interbank_reference':'order_id','auth_code':'codigo_autorizacion',
                             'payworks_reference':'order_id_xpay'}, inplace=True)
sftp['date_file'] = sftp["file_name"].str[-15:-7]
sftp['date_file']= pd.to_datetime(sftp['date_file'], format='%d%m%Y').dt.date
sftp['date_file']= pd.to_datetime(sftp['date_file'])
sftp.sort_values(by=['date_file'],ascending=True,inplace=True)
sftp['month'] = sftp['date_file'].dt.strftime('%B')
sftp.head()

,affiliation_sftp,card,commisions,transaction_date,gross_amount,transaction_type,tax,installments_tax,installments_fees,codigo_autorizacion,order_id,order_id_xpay,acquirer_commision,acquirer_tax,file_name,date_file,month
68324,07927500,4152313796,-61,20220425,4700,SETTLED,-10,0.00,0.00,341078,322244454552,NaN,-61.00,-10.00,PAYCLIP_Settlement_27042022_V1.csv,2022-04-27,April
91499,07927515,4915664400,-80,20220424,6136,SETTLED,-13,0.00,0.00,000026,990396907887,NaN,-80.00,-13.00,PAYCLIP_Settlement_27042022_V1.csv,2022-04-27,April
91498,07927515,4213166192,-642,20220424,49400,SETTLED,-103,0.00,0.00,024868,308872033131,NaN,-642.00,-103.00,PAYCLIP_Settlement_27042022_V1.csv,2022-04-27,April
91497,07927515,6275350184,-48,20220424,3700,SETTLED,-8,0.00,0.00,609195,256071318681,NaN,-48.00,-8.00,PAYCLIP_Settlement_27042022_V1.csv,2022-04-27,April
91496,07927515,4217470023,-137,20220424,10500,SETTLED,-22,0.00,0.00,000786,117980292282,NaN,-137.00,-22.00,PAYCLIP_Settlement_27042022_V1.csv,2022-04-27,April


In [15]:
sftp.shape

(2672202, 17)

In [16]:
sftp_fecha_deposito = sftp[(sftp['transaction_type'] == 'BANORTE')]
sftp_fecha_deposito = sftp_fecha_deposito[['date_file', 'gross_amount']]
sftp_fecha_deposito.rename(columns ={'gross_amount': 'deposit_date'},inplace=True)
sftp_fecha_deposito.head()

,date_file,deposit_date
0,2022-04-27,20220426
0,2022-04-28,20220427
0,2022-04-29,20220428
0,2022-04-30,20220429
0,2022-05-03,20220502


In [17]:
sftp = sftp[(sftp['transaction_type'] != 'BANORTE')]
sftp = sftp.merge(sftp_fecha_deposito[['date_file', 'deposit_date']],on='date_file',how='left')

In [18]:
sftp.head()

,affiliation_sftp,card,commisions,transaction_date,gross_amount,transaction_type,tax,installments_tax,installments_fees,codigo_autorizacion,order_id,order_id_xpay,acquirer_commision,acquirer_tax,file_name,date_file,month,deposit_date
0,07927500,4152313796,-61,20220425,4700,SETTLED,-10,0.00,0.00,341078,322244454552,NaN,-61.00,-10.00,PAYCLIP_Settlement_27042022_V1.csv,2022-04-27,April,20220426
1,07927515,4915664400,-80,20220424,6136,SETTLED,-13,0.00,0.00,000026,990396907887,NaN,-80.00,-13.00,PAYCLIP_Settlement_27042022_V1.csv,2022-04-27,April,20220426
2,07927515,4213166192,-642,20220424,49400,SETTLED,-103,0.00,0.00,024868,308872033131,NaN,-642.00,-103.00,PAYCLIP_Settlement_27042022_V1.csv,2022-04-27,April,20220426
3,07927515,6275350184,-48,20220424,3700,SETTLED,-8,0.00,0.00,609195,256071318681,NaN,-48.00,-8.00,PAYCLIP_Settlement_27042022_V1.csv,2022-04-27,April,20220426
4,07927515,4217470023,-137,20220424,10500,SETTLED,-22,0.00,0.00,000786,117980292282,NaN,-137.00,-22.00,PAYCLIP_Settlement_27042022_V1.csv,2022-04-27,April,20220426


In [19]:
sftp['transaction_date'] = pd.to_datetime(sftp['transaction_date'], format='%Y%m%d').dt.date
sftp['deposit_date'] = pd.to_datetime(sftp['deposit_date'], format='%Y%m%d').dt.date

In [20]:
sftp['transaction_date'] = pd.to_datetime(sftp['transaction_date'])
sftp['deposit_date'] = pd.to_datetime(sftp['deposit_date'])

In [21]:
sftp['gross_amount'] = sftp['gross_amount']/100
sftp['commisions'] = sftp['commisions']/100
sftp['tax'] = sftp['tax']/100
sftp['installments_fees'] = sftp['installments_fees']/100
sftp['installments_tax'] = sftp['installments_tax']/100
sftp['acquirer_commision'] = sftp['acquirer_commision']/100
sftp['acquirer_tax'] = sftp['acquirer_tax']/100
sftp.loc[sftp['transaction_type'] != 'SETTLED', 'gross_amount'] = sftp['gross_amount']*-1
sftp['LAST4_SFTP'] = sftp['card'].str[-4:].astype(str)
sftp['BIN_SFTP'] = sftp['card'].str[:6].astype(str)
sftp['INDEX_TOTAL'] = (sftp['BIN_SFTP'].astype(str).replace(' ', '',regex=True)+'_'
                       +sftp['LAST4_SFTP'].astype(str).replace(' ', '',regex=True)+'_'
                       +sftp['order_id'].astype(str).replace(' ', '',regex=True)+'_'
                       +sftp['gross_amount'].astype(str).replace(' ', '',regex=True))
sftp['INDEX_TOTAL_XPAY'] = (sftp['BIN_SFTP'].astype(str).replace(' ', '',regex=True)+'_'
                       +sftp['LAST4_SFTP'].astype(str).replace(' ', '',regex=True)+'_'
                       +sftp['order_id_xpay'].astype(str).replace(' ', '',regex=True)+'_'
                       +sftp['gross_amount'].astype(str).replace(' ', '',regex=True))
sftp['order_id'].fillna('NO EXISTE EN SFTP',inplace=True)
sftp.head()

,affiliation_sftp,card,commisions,transaction_date,gross_amount,transaction_type,tax,installments_tax,installments_fees,codigo_autorizacion,...,acquirer_commision,acquirer_tax,file_name,date_file,month,deposit_date,LAST4_SFTP,BIN_SFTP,INDEX_TOTAL,INDEX_TOTAL_XPAY
0,07927500,4152313796,-0.61,2022-04-25,47.00,SETTLED,-0.10,0.00,0.00,341078,...,-0.61,-0.10,PAYCLIP_Settlement_27042022_V1.csv,2022-04-27,April,2022-04-26,3796,415231,415231_3796_322244454552_47.0,415231_3796_nan_47.0
1,07927515,4915664400,-0.80,2022-04-24,61.36,SETTLED,-0.13,0.00,0.00,000026,...,-0.80,-0.13,PAYCLIP_Settlement_27042022_V1.csv,2022-04-27,April,2022-04-26,4400,491566,491566_4400_990396907887_61.36,491566_4400_nan_61.36
2,07927515,4213166192,-6.42,2022-04-24,494.00,SETTLED,-1.03,0.00,0.00,024868,...,-6.42,-1.03,PAYCLIP_Settlement_27042022_V1.csv,2022-04-27,April,2022-04-26,6192,421316,421316_6192_308872033131_494.0,421316_6192_nan_494.0
3,07927515,6275350184,-0.48,2022-04-24,37.00,SETTLED,-0.08,0.00,0.00,609195,...,-0.48,-0.08,PAYCLIP_Settlement_27042022_V1.csv,2022-04-27,April,2022-04-26,0184,627535,627535_0184_256071318681_37.0,627535_0184_nan_37.0
4,07927515,4217470023,-1.37,2022-04-24,105.00,SETTLED,-0.22,0.00,0.00,000786,...,-1.37,-0.22,PAYCLIP_Settlement_27042022_V1.csv,2022-04-27,April,2022-04-26,0023,421747,421747_0023_117980292282_105.0,421747_0023_nan_105.0


In [22]:
sftp.shape

(2672196, 22)

### CRUCE

In [23]:
data_status_1 = provision_bnte.merge(sftp[['order_id', 'transaction_date', 'deposit_date','INDEX_TOTAL']],
                                     on='order_id',how='left')
data_status_1['transaction_date'].fillna('NO ENCONTRADO EN SFTP',inplace=True)
data_status_1['deposit_date'].fillna('NO ENCONTRADO EN SFTP',inplace=True)
#data_status_1.drop_duplicates(subset ="INDEX_TOTAL", keep = 'last', inplace = True) 
data_status_1.head()

,merchant_id,transaction_id,order_id,psp,affiliation_type,clave_autorizacion,date_transaction,amount,bin,last4,status_code,msi,clip_reader,report_id,report_date_local,status,CONCAT,transaction_date,deposit_date,INDEX_TOTAL
0,2bb7d4f8-e3d5-4d00-8a48-1330fddbaf29,1a2ad035-dcb1-4b3b-9793-16e27565ff61,562598437233,PROSA,CARD_PRESENT,000047,2022-04-28,137.00,491566,0061,1,0.00,Clip Pro,TPWH4GC2P,2022-04-29,PAID,562598437233-137.0-0061,2022-04-28 00:00:00,2022-04-29 00:00:00,491566_8461_562598437233_137.0
1,183dfdc9-6976-4721-b4b6-142ec26a3c14,bf134325-4700-466d-be69-1d56d0ff1634,321943055354,PROSA,CARD_PRESENT,446966,2022-04-28,67.00,402766,6789,1,0.00,Clip Plus,LT2V7YBTR,2022-04-29,PAID,321943055354-67.0-6789,2022-04-28 00:00:00,2022-04-29 00:00:00,402766_5789_321943055354_67.0
2,529367d5-31de-4c72-8e3c-b0bd75299afe,bc455f37-2a77-4245-94d2-44411ece89d3,892351086541,PROSA,CARD_PRESENT,000468,2022-04-28,90.00,557908,3485,1,0.00,Clip Plus,69Y3SYCRP,2022-04-29,PAID,892351086541-90.0-3485,2022-04-28 00:00:00,2022-04-29 00:00:00,557908_3085_892351086541_90.0
3,f63917ba-4561-4533-88e7-5ab9f3c3248e,e92d391d-2172-4dc5-a96f-e611bedab6ac,537981910143,PROSA,CARD_PRESENT,006024,2022-04-28,73.00,514065,4821,1,0.00,Clip Total,TGCCGW4YB,2022-04-29,PAID,537981910143-73.0-4821,2022-04-28 00:00:00,2022-04-29 00:00:00,514065_0021_537981910143_73.0
4,406e0bef-bb42-452e-9056-5522662d1991,67c1271e-b4c9-4da8-af85-bbcd8bbe31b6,830007589726,PROSA,CARD_PRESENT,994592,2022-04-28,200.00,542878,1133,1,0.00,Clip Plus 2,NHH4KMBG8,2022-04-29,PAID,830007589726-200.0-1133,2022-04-28 00:00:00,2022-04-29 00:00:00,542878_0233_830007589726_200.0


In [24]:
data_status_1.shape

(1184981, 20)

In [25]:
duplicates_status_1 = data_status_1[data_status_1.duplicated(['INDEX_TOTAL'] , keep=False)]
duplicates_status_1.head()

,merchant_id,transaction_id,order_id,psp,affiliation_type,clave_autorizacion,date_transaction,amount,bin,last4,status_code,msi,clip_reader,report_id,report_date_local,status,CONCAT,transaction_date,deposit_date,INDEX_TOTAL
2709,7236daa9-c9a2-41ba-9eac-799778fd8f98,e9594d4a-840e-4782-832e-9466b5bd809a,473462389622,PROSA,CARD_PRESENT,004438,2022-04-28,83.00,480077,7732,1,0.00,Clip Total,89QXQRJ8T,2022-04-29,PAID,473462389622-83.0-7732,NO ENCONTRADO EN SFTP,NO ENCONTRADO EN SFTP,NaN
3456,f80b2edf-6053-41f8-b05e-f5a6566f21a8,78349720-bd30-484d-97de-760026259724,659089359426,PROSA,CARD_PRESENT,006741,2022-04-28,35.00,480077,6878,1,0.00,Clip Total,ZN5R43835,2022-04-29,PAID,659089359426-35.0-6878,NO ENCONTRADO EN SFTP,NO ENCONTRADO EN SFTP,NaN
6652,cb5f7319-6330-4fd5-8295-ee3c3ac8d3f9,a2308f25-b61a-4efc-831d-f667c7ac83c6,096544913938,PROSA,CARD_PRESENT,728256,2022-04-28,"3,450.00",636318,8301,1,0.00,Clip Plus 2,9CSTD6M5X,2022-04-29,PAID,096544913938-3450.0-8301,NO ENCONTRADO EN SFTP,NO ENCONTRADO EN SFTP,NaN
6668,5264346e-c639-43a3-a37c-aa2bc11f9cc7,a87631b6-d35a-45b7-a058-5bee58ecd1d8,512107180414,PROSA,CARD_PRESENT,695696,2022-04-28,565.00,549949,5526,1,0.00,Clip Pro,Q49ZP2S4V,2022-04-29,PAID,512107180414-565.0-5526,NO ENCONTRADO EN SFTP,NO ENCONTRADO EN SFTP,NaN
6771,128fe03f-d7d9-4343-bd30-6448bd4562cf,0a81a442-2cba-4527-84ac-92cee15009e1,642763643806,PROSA,CARD_PRESENT,949721,2022-04-28,78.00,547074,0415,1,0.00,Clip Plus 2,NBFN76P6P,2022-04-29,PAID,642763643806-78.0-0415,NO ENCONTRADO EN SFTP,NO ENCONTRADO EN SFTP,NaN


In [26]:
duplicates_status_1.shape

(267, 20)

In [27]:
results_status_1 = data_status_1[(data_status_1['transaction_date'] != 'NO ENCONTRADO EN SFTP')]
results_status_1.head()

,merchant_id,transaction_id,order_id,psp,affiliation_type,clave_autorizacion,date_transaction,amount,bin,last4,status_code,msi,clip_reader,report_id,report_date_local,status,CONCAT,transaction_date,deposit_date,INDEX_TOTAL
0,2bb7d4f8-e3d5-4d00-8a48-1330fddbaf29,1a2ad035-dcb1-4b3b-9793-16e27565ff61,562598437233,PROSA,CARD_PRESENT,000047,2022-04-28,137.00,491566,0061,1,0.00,Clip Pro,TPWH4GC2P,2022-04-29,PAID,562598437233-137.0-0061,2022-04-28 00:00:00,2022-04-29 00:00:00,491566_8461_562598437233_137.0
1,183dfdc9-6976-4721-b4b6-142ec26a3c14,bf134325-4700-466d-be69-1d56d0ff1634,321943055354,PROSA,CARD_PRESENT,446966,2022-04-28,67.00,402766,6789,1,0.00,Clip Plus,LT2V7YBTR,2022-04-29,PAID,321943055354-67.0-6789,2022-04-28 00:00:00,2022-04-29 00:00:00,402766_5789_321943055354_67.0
2,529367d5-31de-4c72-8e3c-b0bd75299afe,bc455f37-2a77-4245-94d2-44411ece89d3,892351086541,PROSA,CARD_PRESENT,000468,2022-04-28,90.00,557908,3485,1,0.00,Clip Plus,69Y3SYCRP,2022-04-29,PAID,892351086541-90.0-3485,2022-04-28 00:00:00,2022-04-29 00:00:00,557908_3085_892351086541_90.0
3,f63917ba-4561-4533-88e7-5ab9f3c3248e,e92d391d-2172-4dc5-a96f-e611bedab6ac,537981910143,PROSA,CARD_PRESENT,006024,2022-04-28,73.00,514065,4821,1,0.00,Clip Total,TGCCGW4YB,2022-04-29,PAID,537981910143-73.0-4821,2022-04-28 00:00:00,2022-04-29 00:00:00,514065_0021_537981910143_73.0
4,406e0bef-bb42-452e-9056-5522662d1991,67c1271e-b4c9-4da8-af85-bbcd8bbe31b6,830007589726,PROSA,CARD_PRESENT,994592,2022-04-28,200.00,542878,1133,1,0.00,Clip Plus 2,NHH4KMBG8,2022-04-29,PAID,830007589726-200.0-1133,2022-04-28 00:00:00,2022-04-29 00:00:00,542878_0233_830007589726_200.0


In [28]:
results_status_1.shape

(1184722, 20)

In [29]:
NA_status_1 = data_status_1[(data_status_1['transaction_date'] == 'NO ENCONTRADO EN SFTP')]
NA_status_1.head()

,merchant_id,transaction_id,order_id,psp,affiliation_type,clave_autorizacion,date_transaction,amount,bin,last4,status_code,msi,clip_reader,report_id,report_date_local,status,CONCAT,transaction_date,deposit_date,INDEX_TOTAL
2709,7236daa9-c9a2-41ba-9eac-799778fd8f98,e9594d4a-840e-4782-832e-9466b5bd809a,473462389622,PROSA,CARD_PRESENT,004438,2022-04-28,83.00,480077,7732,1,0.00,Clip Total,89QXQRJ8T,2022-04-29,PAID,473462389622-83.0-7732,NO ENCONTRADO EN SFTP,NO ENCONTRADO EN SFTP,NaN
3456,f80b2edf-6053-41f8-b05e-f5a6566f21a8,78349720-bd30-484d-97de-760026259724,659089359426,PROSA,CARD_PRESENT,006741,2022-04-28,35.00,480077,6878,1,0.00,Clip Total,ZN5R43835,2022-04-29,PAID,659089359426-35.0-6878,NO ENCONTRADO EN SFTP,NO ENCONTRADO EN SFTP,NaN
6652,cb5f7319-6330-4fd5-8295-ee3c3ac8d3f9,a2308f25-b61a-4efc-831d-f667c7ac83c6,096544913938,PROSA,CARD_PRESENT,728256,2022-04-28,"3,450.00",636318,8301,1,0.00,Clip Plus 2,9CSTD6M5X,2022-04-29,PAID,096544913938-3450.0-8301,NO ENCONTRADO EN SFTP,NO ENCONTRADO EN SFTP,NaN
6668,5264346e-c639-43a3-a37c-aa2bc11f9cc7,a87631b6-d35a-45b7-a058-5bee58ecd1d8,512107180414,PROSA,CARD_PRESENT,695696,2022-04-28,565.00,549949,5526,1,0.00,Clip Pro,Q49ZP2S4V,2022-04-29,PAID,512107180414-565.0-5526,NO ENCONTRADO EN SFTP,NO ENCONTRADO EN SFTP,NaN
6771,128fe03f-d7d9-4343-bd30-6448bd4562cf,0a81a442-2cba-4527-84ac-92cee15009e1,642763643806,PROSA,CARD_PRESENT,949721,2022-04-28,78.00,547074,0415,1,0.00,Clip Plus 2,NBFN76P6P,2022-04-29,PAID,642763643806-78.0-0415,NO ENCONTRADO EN SFTP,NO ENCONTRADO EN SFTP,NaN


In [30]:
NA_status_1.shape

(259, 20)

### DATA FINAL

In [31]:
provision_prosa_final = pd.concat([results_status_1,NA_status_1])
provision_prosa_final['status'].fillna('RETAINED',inplace=True)
provision_prosa_final['report_date_local'].fillna('NO ENCONTRADO',inplace=True)
provision_prosa_final.head()

,merchant_id,transaction_id,order_id,psp,affiliation_type,clave_autorizacion,date_transaction,amount,bin,last4,status_code,msi,clip_reader,report_id,report_date_local,status,CONCAT,transaction_date,deposit_date,INDEX_TOTAL
0,2bb7d4f8-e3d5-4d00-8a48-1330fddbaf29,1a2ad035-dcb1-4b3b-9793-16e27565ff61,562598437233,PROSA,CARD_PRESENT,000047,2022-04-28,137.00,491566,0061,1,0.00,Clip Pro,TPWH4GC2P,2022-04-29,PAID,562598437233-137.0-0061,2022-04-28 00:00:00,2022-04-29 00:00:00,491566_8461_562598437233_137.0
1,183dfdc9-6976-4721-b4b6-142ec26a3c14,bf134325-4700-466d-be69-1d56d0ff1634,321943055354,PROSA,CARD_PRESENT,446966,2022-04-28,67.00,402766,6789,1,0.00,Clip Plus,LT2V7YBTR,2022-04-29,PAID,321943055354-67.0-6789,2022-04-28 00:00:00,2022-04-29 00:00:00,402766_5789_321943055354_67.0
2,529367d5-31de-4c72-8e3c-b0bd75299afe,bc455f37-2a77-4245-94d2-44411ece89d3,892351086541,PROSA,CARD_PRESENT,000468,2022-04-28,90.00,557908,3485,1,0.00,Clip Plus,69Y3SYCRP,2022-04-29,PAID,892351086541-90.0-3485,2022-04-28 00:00:00,2022-04-29 00:00:00,557908_3085_892351086541_90.0
3,f63917ba-4561-4533-88e7-5ab9f3c3248e,e92d391d-2172-4dc5-a96f-e611bedab6ac,537981910143,PROSA,CARD_PRESENT,006024,2022-04-28,73.00,514065,4821,1,0.00,Clip Total,TGCCGW4YB,2022-04-29,PAID,537981910143-73.0-4821,2022-04-28 00:00:00,2022-04-29 00:00:00,514065_0021_537981910143_73.0
4,406e0bef-bb42-452e-9056-5522662d1991,67c1271e-b4c9-4da8-af85-bbcd8bbe31b6,830007589726,PROSA,CARD_PRESENT,994592,2022-04-28,200.00,542878,1133,1,0.00,Clip Plus 2,NHH4KMBG8,2022-04-29,PAID,830007589726-200.0-1133,2022-04-28 00:00:00,2022-04-29 00:00:00,542878_0233_830007589726_200.0


In [32]:
provision_prosa_final.shape

(1184981, 20)

In [33]:
grouped = provision_prosa_final.groupby(provision_prosa_final.date_transaction)
grouped

In [34]:
group_1 = grouped.get_group('2022-04-28')
group_2 = grouped.get_group('2022-04-29')
group_3 = grouped.get_group('2022-04-30')

### PIVOT

#### REPORT_DATE_LOCAL

In [35]:
pivot_bnte_report_date_local = pd.pivot_table(provision_prosa_final,index=['date_transaction']
                                              ,columns=['report_date_local'],values=['amount']
                                              ,aggfunc=np.sum,margins=True).reset_index()
pivot_bnte_report_date_local.fillna(0,inplace=True)
pivot_bnte_report_date_local.head()

date_transaction       amount                                \
report_date_local                    2022-04-28     2022-04-29     2022-04-30   
0                       2022-04-28 2,483,487.98 153,003,622.78     872,245.75   
1                       2022-04-29         0.00   3,059,189.43 191,194,167.79   
2                       2022-04-30         0.00           0.00   3,269,995.20   
3                              All 2,483,487.98 156,062,812.21 195,336,408.74   

                                                                          
report_date_local     2022-05-01 2022-05-02 NO ENCONTRADO            All  
0                           0.00       0.00          0.00 156,359,356.51  
1                     960,839.24       0.00          2.90 195,214,199.36  
2                 217,257,639.23 696,827.69          0.00 221,224,462.12  
3                 218,218,478.47 696,827.69          2.90 572,798,017.99

In [36]:
pivot_bnte_report_date_local.shape

(4, 8)

#### FECHA_DEPOSITO

In [37]:
pivot_bnte_fecha_deposito = pd.pivot_table(provision_prosa_final,index=['date_transaction']
                                              ,columns=['deposit_date'],values=['amount']
                                              ,aggfunc=np.sum,margins=True).reset_index()
pivot_bnte_fecha_deposito.fillna(0,inplace=True)
pivot_bnte_fecha_deposito.head()

date_transaction              amount                      \
deposit_date                  2022-04-27 00:00:00 2022-04-29 00:00:00   
0                  2022-04-28                0.00      151,564,669.43   
1                  2022-04-29                0.00                0.00   
2                  2022-04-30              535.00              109.00   
3                         All              535.00      151,564,778.43   

                                                                            \
deposit_date 2022-05-02 00:00:00 2022-05-03 00:00:00 NO ENCONTRADO EN SFTP   
0                   4,776,318.22                0.00             18,368.86   
1                 195,192,675.15              271.00             21,253.21   
2                 221,191,770.01                0.00             32,048.11   
3                 421,160,763.38              271.00             71,670.18   

                             
deposit_date            All  
0            156,359,356.51  
1            195,214,199.36  
2            221,224,462.12  
3            572,798,017.99

In [38]:
pivot_bnte_fecha_deposito.shape

(4, 7)

#### READER

In [39]:
pivot_reader_bnte = pd.pivot_table(provision_prosa_final,index=['deposit_date','clip_reader']
                                              ,columns=['report_date_local'],values=['amount']
                                              ,aggfunc=np.sum,margins=True).reset_index(1)
pivot_reader_bnte.fillna(0,inplace=True)
pivot_reader_bnte.head()

clip_reader       amount                           \
report_date_local                   2022-04-28    2022-04-29 2022-04-30   
deposit_date                                                              
2022-04-27 00:00:00   Clip Plus 2         0.00          0.00       0.00   
2022-04-29 00:00:00  Clip Classic   258,224.46  6,278,276.06 849,796.75   
2022-04-29 00:00:00     Clip Plus   431,202.82 31,082,923.95       0.00   
2022-04-29 00:00:00   Clip Plus 2 1,031,553.60 48,624,835.68       0.00   
2022-04-29 00:00:00      Clip Pro   279,226.99 26,162,348.77       0.00   

                                                                       
report_date_local   2022-05-01 2022-05-02 NO ENCONTRADO           All  
deposit_date                                                           
2022-04-27 00:00:00     535.00       0.00          0.00        535.00  
2022-04-29 00:00:00       0.00       0.00          0.00  7,386,297.27  
2022-04-29 00:00:00       0.00       0.00          0.00 31,514,126.77  
2022-04-29 00:00:00       0.00       0.00          0.00 49,656,389.28  
2022-04-29 00:00:00       0.00       0.00          0.00 26,441,575.76

In [40]:
pivot_reader_bnte.shape

(18, 8)

In [41]:
# writer = pd.ExcelWriter('SF PROVISIONES SEPTIEMBRE 2020.xlsx', engine='xlsxwriter', options={'encoding':'utf-8'})
# provision_prosa_final.to_excel(writer, sheet_name='SEPTIEMBRE BNTE', index=False)
# pivot_bnte_report_date_local.to_excel(writer, sheet_name='PIVOT BNTE')
# writer.save()

# PROVISIONES AMEX

In [42]:
provision_amex = all_provisiones[(all_provisiones['psp'] == 'AMEX')]
provision_amex['order_id'] = provision_amex['order_id'].str.upper() 
provision_amex['CONCAT'] = (provision_amex['order_id'].astype(str).replace(' ', '',regex=True)+'-'
                            +provision_amex['amount'].apply(np.floor).astype(str).replace(' ', '',regex=True)+'-'
                            +provision_amex['last4'].astype(str).replace(' ', '',regex=True))
provision_amex.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_13792\4019274069.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  provision_amex['order_id'] = provision_amex['order_id'].str.upper()
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_13792\4019274069.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  provision_amex['CONCAT'] = (provision_amex['order_id'].astype(str).replace(' ', '',regex=True)+'-'


,merchant_id,transaction_id,order_id,psp,affiliation_type,clave_autorizacion,date_transaction,amount,bin,last4,status_code,msi,clip_reader,report_id,report_date_local,status,CONCAT
30,88615f3b-6b3e-4e57-8604-b83fffc6da75,38312d42-3784-482b-81d9-551ebf9b7113,FGTWH5CLJH7Q,AMEX,OPTBLUE,001501876279486,2022-04-28,10.00,375150,6778,1,0.00,Clip Plus 2,CZ3BFZZNK,2022-04-29,PAID,FGTWH5CLJH7Q-10.0-6778
42,f1395f50-916d-446b-b211-7d255696f51a,e015225b-01a6-4d61-a4bd-c75e38eb1fb4,ZE7DKJZOLXQ7,AMEX,OPTBLUE,001501873842484,2022-04-28,744.00,376702,1006,1,0.00,Clip Pro,M7CH6L4KB,2022-04-29,PAID,ZE7DKJZOLXQ7-744.0-1006
58,cd160dcf-d9d9-4453-9190-c79e221780b4,ef1dd50a-bb1c-4f8d-a22d-a87c37c6c046,CHD2BOI5LFYT,AMEX,OPTBLUE,001501879142483,2022-04-28,588.50,371776,1006,1,0.00,Clip Total,6V28TYBXT,2022-04-29,PAID,CHD2BOI5LFYT-588.0-1006
104,c2e0bdf0-e3b1-48b2-affc-2bc1e5e035a6,13a92d7b-1edd-43c0-85ca-bdac489e209b,SDWEASRTEAJG,AMEX,OPTBLUE,001501876284486,2022-04-28,300.00,371773,1000,1,0.00,Clip Total,XS7JF3FZ2,2022-04-29,PAID,SDWEASRTEAJG-300.0-1000
119,4183a913-fbfe-4eeb-bde8-99da3a109af0,bd22a45c-0769-4467-a0a9-a47ef85b68c7,QQKDOWUFNMFY,AMEX,OPTBLUE,001501877262481,2022-04-28,"3,979.00",376679,4005,1,0.00,Clip Pro,92GQ46QH8,2022-04-29,PAID,QQKDOWUFNMFY-3979.0-4005


In [43]:
provision_amex.shape

(22972, 17)

### RUTAS DE ACCESO EPA

In [44]:
epa = pd.read_csv(r'G:\My Drive\3.-Settlement\PROVISIONES\2022\04. ABRIL\SFTP\AMEX\AMEX PROVISION EPA ABRIL 2022.csv')   
epa.head()


,order_id,Type,Date_process_roc,A,Total_amount_charge_roc,Svc_fee_amount_roc,Svc_fee_amount_accel_roc,roc_deposit_date,15,rechazo_amex,total_amount_bruto,fee_amount_mdr,fee_amount_msi,iva_msi,iva_mdr,bin_bank,last4_bank,INDEX_TOTAL
0,MQUPW4E2ZVER,4,2022-04-20,379660711212005****,"42,000.00",0.00,0.00,2022-04-22,0,0,420.00,-10.08,0.00,0.00,-1.61,379660,2005,379660_2005_MQUPW4E2ZVER_420.0
1,JXXZ6ASP1XGN,4,2022-04-20,379180723071349****,"29,400.00",0.00,0.00,2022-04-22,0,0,294.00,-7.06,0.00,0.00,-1.13,379180,1349,379180_1349_JXXZ6ASP1XGN_294.0
2,XKIPAGBMXEJ6,4,2022-04-21,376662350074004****,"150,000.00",0.00,"-3,750.00",2022-04-26,3,0,"1,500.00",-36.00,0.00,0.00,-5.76,376662,4004,376662_4004_XKIPAGBMXEJ6_1500.0
3,JMCGJH1ZK6VI,4,2022-04-21,376700385381018****,"520,900.00",0.00,0.00,2022-04-26,0,0,"5,209.00",-125.02,0.00,0.00,-20.00,376700,1018,376700_1018_JMCGJH1ZK6VI_5209.0
4,Q2BTPN2RTCIN,4,2022-04-21,376700257641002****,"10,000.00",0.00,0.00,2022-04-26,0,0,100.00,-2.40,0.00,0.00,-0.38,376700,1002,376700_1002_Q2BTPN2RTCIN_100.0


In [45]:
epa.shape

(291, 18)

In [46]:
epa.columns

Index(['order_id', 'Type', 'Date_process_roc', 'A', 'Total_amount_charge_roc',
       'Svc_fee_amount_roc', 'Svc_fee_amount_accel_roc', 'roc_deposit_date',
       '15', 'rechazo_amex', 'total_amount_bruto', 'fee_amount_mdr',
       'fee_amount_msi', 'iva_msi', 'iva_mdr', 'bin_bank', 'last4_bank',
       'INDEX_TOTAL'],
      dtype='object')

### DATA FINAL

In [47]:
provision_amex_1 = provision_amex.merge(epa[['order_id','Date_process_roc','roc_deposit_date']]
                                        ,on='order_id',how='left')
#provision_amex_final.fillna('RETAINED',inplace=True)
provision_amex_1.rename(columns ={'Date_process_roc': 'date_process','roc_deposit_date': 'deposit_date'},inplace=True)
provision_amex_1['date_process'].fillna('NO ENCONTRADO EN EPA',inplace=True)
provision_amex_1['deposit_date'].fillna('NO ENCONTRADO EN EPA',inplace=True)
provision_amex_1.head()

,merchant_id,transaction_id,order_id,psp,affiliation_type,clave_autorizacion,date_transaction,amount,bin,last4,status_code,msi,clip_reader,report_id,report_date_local,status,CONCAT,date_process,deposit_date
0,88615f3b-6b3e-4e57-8604-b83fffc6da75,38312d42-3784-482b-81d9-551ebf9b7113,FGTWH5CLJH7Q,AMEX,OPTBLUE,001501876279486,2022-04-28,10.00,375150,6778,1,0.00,Clip Plus 2,CZ3BFZZNK,2022-04-29,PAID,FGTWH5CLJH7Q-10.0-6778,NO ENCONTRADO EN EPA,NO ENCONTRADO EN EPA
1,f1395f50-916d-446b-b211-7d255696f51a,e015225b-01a6-4d61-a4bd-c75e38eb1fb4,ZE7DKJZOLXQ7,AMEX,OPTBLUE,001501873842484,2022-04-28,744.00,376702,1006,1,0.00,Clip Pro,M7CH6L4KB,2022-04-29,PAID,ZE7DKJZOLXQ7-744.0-1006,NO ENCONTRADO EN EPA,NO ENCONTRADO EN EPA
2,cd160dcf-d9d9-4453-9190-c79e221780b4,ef1dd50a-bb1c-4f8d-a22d-a87c37c6c046,CHD2BOI5LFYT,AMEX,OPTBLUE,001501879142483,2022-04-28,588.50,371776,1006,1,0.00,Clip Total,6V28TYBXT,2022-04-29,PAID,CHD2BOI5LFYT-588.0-1006,NO ENCONTRADO EN EPA,NO ENCONTRADO EN EPA
3,c2e0bdf0-e3b1-48b2-affc-2bc1e5e035a6,13a92d7b-1edd-43c0-85ca-bdac489e209b,SDWEASRTEAJG,AMEX,OPTBLUE,001501876284486,2022-04-28,300.00,371773,1000,1,0.00,Clip Total,XS7JF3FZ2,2022-04-29,PAID,SDWEASRTEAJG-300.0-1000,NO ENCONTRADO EN EPA,NO ENCONTRADO EN EPA
4,4183a913-fbfe-4eeb-bde8-99da3a109af0,bd22a45c-0769-4467-a0a9-a47ef85b68c7,QQKDOWUFNMFY,AMEX,OPTBLUE,001501877262481,2022-04-28,"3,979.00",376679,4005,1,0.00,Clip Pro,92GQ46QH8,2022-04-29,PAID,QQKDOWUFNMFY-3979.0-4005,NO ENCONTRADO EN EPA,NO ENCONTRADO EN EPA


In [48]:
provision_amex_1.shape

(22972, 19)

In [49]:
results_amex_final = provision_amex_1[(provision_amex_1['deposit_date'] != 'NO ENCONTRADO EN EPA')]
results_amex_final.head()

,merchant_id,transaction_id,order_id,psp,affiliation_type,clave_autorizacion,date_transaction,amount,bin,last4,status_code,msi,clip_reader,report_id,report_date_local,status,CONCAT,date_process,deposit_date
502,17bf442a-7d5c-41fc-a21a-07205119ddec,7f4d7de2-54ae-4bc9-907b-e3bf932803f2,BTQOXDTZKTHU,AMEX,AGGREGATOR,001501950562484,2022-04-28,40.00,376724,1049,1,0.00,Clip Pro,3FH67DDML,2022-04-29,PAID,BTQOXDTZKTHU-40.0-1049,2022-04-28,2022-05-03
513,d48ea647-f256-4cd5-ae01-e7279b1d998d,e921defe-4f16-4fe8-b5f7-7bd0dcef8911,P3DCTGE9Y9PQ,AMEX,AGGREGATOR,001501950654489,2022-04-28,"3,907.62",370783,7018,1,0.00,Clip Plus,C32GQQDBQ,2022-04-29,PAID,P3DCTGE9Y9PQ-3907.0-7018,2022-04-28,2022-05-03
571,3bdf6fa0-adf4-4d76-8d3f-3d710407219f,a2a1082b-4562-46fc-9f32-9a747a297c83,9ZMB5LC5KGK5,AMEX,AGGREGATOR,001501961208481,2022-04-28,55.00,376700,1008,1,0.00,Clip Total,4D9LC9Z54,2022-04-29,PAID,9ZMB5LC5KGK5-55.0-1008,2022-04-28,2022-05-03
977,3bdf6fa0-adf4-4d76-8d3f-3d710407219f,153055dd-b107-4176-b4c6-c966d43b671e,JCYDI0HFDATB,AMEX,AGGREGATOR,001502004577486,2022-04-28,220.00,376703,1025,1,0.00,Clip Total,4D9LC9Z54,2022-04-29,PAID,JCYDI0HFDATB-220.0-1025,2022-04-28,2022-05-03
1183,2f97f8c9-8851-4e29-a47b-52de9c12cf93,7ca8cfbd-9b66-492d-8b4d-59d7fcd56a09,DZQYIY1MVN2N,AMEX,AGGREGATOR,001502026958482,2022-04-28,735.00,371710,9001,1,0.00,Clip Plus 2,SXTPJ7P96,2022-04-29,PAID,DZQYIY1MVN2N-735.0-9001,2022-04-28,2022-05-03


In [50]:
results_amex_final.shape

(76, 19)

In [51]:
results_amex_final['date_process'] = pd.to_datetime(results_amex_final['date_process'])
results_amex_final['deposit_date'] = pd.to_datetime(results_amex_final['deposit_date'])

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_13792\2158752409.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_amex_final['date_process'] = pd.to_datetime(results_amex_final['date_process'])
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_13792\2158752409.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_amex_final['deposit_date'] = pd.to_datetime(results_amex_final['deposit_date'])


In [52]:
results_amex_final.head()

,merchant_id,transaction_id,order_id,psp,affiliation_type,clave_autorizacion,date_transaction,amount,bin,last4,status_code,msi,clip_reader,report_id,report_date_local,status,CONCAT,date_process,deposit_date
502,17bf442a-7d5c-41fc-a21a-07205119ddec,7f4d7de2-54ae-4bc9-907b-e3bf932803f2,BTQOXDTZKTHU,AMEX,AGGREGATOR,001501950562484,2022-04-28,40.00,376724,1049,1,0.00,Clip Pro,3FH67DDML,2022-04-29,PAID,BTQOXDTZKTHU-40.0-1049,2022-04-28,2022-05-03
513,d48ea647-f256-4cd5-ae01-e7279b1d998d,e921defe-4f16-4fe8-b5f7-7bd0dcef8911,P3DCTGE9Y9PQ,AMEX,AGGREGATOR,001501950654489,2022-04-28,"3,907.62",370783,7018,1,0.00,Clip Plus,C32GQQDBQ,2022-04-29,PAID,P3DCTGE9Y9PQ-3907.0-7018,2022-04-28,2022-05-03
571,3bdf6fa0-adf4-4d76-8d3f-3d710407219f,a2a1082b-4562-46fc-9f32-9a747a297c83,9ZMB5LC5KGK5,AMEX,AGGREGATOR,001501961208481,2022-04-28,55.00,376700,1008,1,0.00,Clip Total,4D9LC9Z54,2022-04-29,PAID,9ZMB5LC5KGK5-55.0-1008,2022-04-28,2022-05-03
977,3bdf6fa0-adf4-4d76-8d3f-3d710407219f,153055dd-b107-4176-b4c6-c966d43b671e,JCYDI0HFDATB,AMEX,AGGREGATOR,001502004577486,2022-04-28,220.00,376703,1025,1,0.00,Clip Total,4D9LC9Z54,2022-04-29,PAID,JCYDI0HFDATB-220.0-1025,2022-04-28,2022-05-03
1183,2f97f8c9-8851-4e29-a47b-52de9c12cf93,7ca8cfbd-9b66-492d-8b4d-59d7fcd56a09,DZQYIY1MVN2N,AMEX,AGGREGATOR,001502026958482,2022-04-28,735.00,371710,9001,1,0.00,Clip Plus 2,SXTPJ7P96,2022-04-29,PAID,DZQYIY1MVN2N-735.0-9001,2022-04-28,2022-05-03


In [53]:
results_amex_final.shape

(76, 19)

### PIVOT

#### REPORT_DATE_LOCAL

In [54]:
pivot_amex_report_date_local = pd.pivot_table(results_amex_final,index=['date_transaction']
                                              ,columns=['report_date_local'],values=['amount']
                                              ,aggfunc=np.sum,margins=True).reset_index()
pivot_amex_report_date_local.fillna(0,inplace=True)
pivot_amex_report_date_local.head()

date_transaction     amount                        \
report_date_local                  2022-04-29 2022-04-30 2022-05-01   
0                       2022-04-28  37,568.12       0.00       0.00   
1                       2022-04-29       0.00  58,630.18       0.00   
2                       2022-04-30       0.00       0.00  75,511.50   
3                              All  37,568.12  58,630.18  75,511.50   

                                         
report_date_local 2022-05-02        All  
0                       0.00  37,568.12  
1                       0.00  58,630.18  
2                     200.00  75,711.50  
3                     200.00 171,909.80

In [55]:
pivot_amex_report_date_local.shape

(4, 6)

#### FECHA_DEPOSITO

In [56]:
pivot_amex_fecha_deposito = pd.pivot_table(results_amex_final,index=['date_transaction']
                                              ,columns=['deposit_date'],values=['amount']
                                              ,aggfunc=np.sum,margins=True).reset_index()
pivot_amex_fecha_deposito.fillna(0,inplace=True)
pivot_amex_fecha_deposito.head()

date_transaction              amount                      \
deposit_date                  2022-05-03 00:00:00 2022-05-04 00:00:00   
0                  2022-04-28           37,568.12                0.00   
1                  2022-04-29           58,630.18                0.00   
2                  2022-04-30           74,601.50            1,110.00   
3                         All          170,799.80            1,110.00   

                         
deposit_date        All  
0             37,568.12  
1             58,630.18  
2             75,711.50  
3            171,909.80

In [57]:
pivot_amex_fecha_deposito.shape

(4, 4)

#### READER

In [58]:
pivot_reader_amex = pd.pivot_table(results_amex_final,index=['deposit_date','clip_reader']
                                              ,columns=['report_date_local'],values=['amount']
                                              ,aggfunc=np.sum,margins=True).reset_index(1)
pivot_reader_amex.fillna(0,inplace=True)
pivot_reader_amex.head()

clip_reader     amount                                   \
report_date_local                 2022-04-29 2022-04-30 2022-05-01 2022-05-02   
deposit_date                                                                    
2022-05-03 00:00:00  Clip Classic   8,750.00       0.00   8,828.00     200.00   
2022-05-03 00:00:00     Clip Plus  26,418.12  43,551.15  10,802.50       0.00   
2022-05-03 00:00:00   Clip Plus 2   1,027.00  10,105.03  48,015.00       0.00   
2022-05-03 00:00:00      Clip Pro     997.00     908.00   6,008.00       0.00   
2022-05-03 00:00:00    Clip Total     376.00   4,066.00     748.00       0.00   

                               
report_date_local         All  
deposit_date                   
2022-05-03 00:00:00 17,778.00  
2022-05-03 00:00:00 80,771.77  
2022-05-03 00:00:00 59,147.03  
2022-05-03 00:00:00  7,913.00  
2022-05-03 00:00:00  5,190.00

In [59]:
pivot_reader_amex.shape

(7, 6)

In [60]:
# book = load_workbook('SF PROVISIONES SEPTIEMBRE 2020.xlsx')
# writer = pd.ExcelWriter('SF PROVISIONES SEPTIEMBRE 2020.xlsx', engine='openpyxl', options={'encoding':'utf-8'})
# writer.book = book
# provision_amex_final.to_excel(writer, sheet_name='SEPTIEMBRE AMEX', index=False)
# writer.save()
# writer.close()

# PROVISIONES OPTBLUE

In [61]:
provision_optb = provision_amex_1[(provision_amex_1['deposit_date'] == 'NO ENCONTRADO EN EPA')]
provision_optb['order_id'] = provision_optb['order_id'].str.upper() 
provision_optb['CONCAT'] = provision_optb['order_id'].astype(str).replace(' ', '',regex=True)+'-'+provision_optb['amount'].apply(np.floor).astype(str).replace(' ', '',regex=True)+'-'+provision_optb['last4'].astype(str).replace(' ', '',regex=True)
provision_optb.drop(['date_process','deposit_date'],axis=1,inplace=True)
provision_optb.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_13792\1663497610.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  provision_optb['order_id'] = provision_optb['order_id'].str.upper()
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_13792\1663497610.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  provision_optb['CONCAT'] = provision_optb['order_id'].astype(str).replace(' ', '',regex=True)+'-'+provision_optb['amount'].apply(np.floor).astype(str).replace(' ', '',regex=True)+'-'+provision_o

,merchant_id,transaction_id,order_id,psp,affiliation_type,clave_autorizacion,date_transaction,amount,bin,last4,status_code,msi,clip_reader,report_id,report_date_local,status,CONCAT
0,88615f3b-6b3e-4e57-8604-b83fffc6da75,38312d42-3784-482b-81d9-551ebf9b7113,FGTWH5CLJH7Q,AMEX,OPTBLUE,001501876279486,2022-04-28,10.00,375150,6778,1,0.00,Clip Plus 2,CZ3BFZZNK,2022-04-29,PAID,FGTWH5CLJH7Q-10.0-6778
1,f1395f50-916d-446b-b211-7d255696f51a,e015225b-01a6-4d61-a4bd-c75e38eb1fb4,ZE7DKJZOLXQ7,AMEX,OPTBLUE,001501873842484,2022-04-28,744.00,376702,1006,1,0.00,Clip Pro,M7CH6L4KB,2022-04-29,PAID,ZE7DKJZOLXQ7-744.0-1006
2,cd160dcf-d9d9-4453-9190-c79e221780b4,ef1dd50a-bb1c-4f8d-a22d-a87c37c6c046,CHD2BOI5LFYT,AMEX,OPTBLUE,001501879142483,2022-04-28,588.50,371776,1006,1,0.00,Clip Total,6V28TYBXT,2022-04-29,PAID,CHD2BOI5LFYT-588.0-1006
3,c2e0bdf0-e3b1-48b2-affc-2bc1e5e035a6,13a92d7b-1edd-43c0-85ca-bdac489e209b,SDWEASRTEAJG,AMEX,OPTBLUE,001501876284486,2022-04-28,300.00,371773,1000,1,0.00,Clip Total,XS7JF3FZ2,2022-04-29,PAID,SDWEASRTEAJG-300.0-1000
4,4183a913-fbfe-4eeb-bde8-99da3a109af0,bd22a45c-0769-4467-a0a9-a47ef85b68c7,QQKDOWUFNMFY,AMEX,OPTBLUE,001501877262481,2022-04-28,"3,979.00",376679,4005,1,0.00,Clip Pro,92GQ46QH8,2022-04-29,PAID,QQKDOWUFNMFY-3979.0-4005


In [62]:
provision_optb.shape

(22896, 17)

### RUTAS DE ACCERSO GRRCN 

In [63]:
opt = pd.read_csv(r'G:\My Drive\3.-Settlement\PROVISIONES\2022\04. ABRIL\SFTP\AMEX\AMEX PROVISION OPTBLUE ABRIL 2022.csv')   
opt.head()


,1,2,3,4,fecha_pago,6,numero_establecimiento,fecha_presentacion,fecha_procesamiento,10,...,Monto_primera_cuota,Monto_cuota_siguiente,Monto_servicio,Monto_aceleracion,last4_bank,bin_bank,INDEX_TOTAL,INDEX_PARCIAL,index_comisiones,index_comisiones_industria
0,TRANSACTN,8138352885,2,1180082360,2022-04-29,MXN,8138464136,2022-04-28,2022-04-28,133119,...,0.00,0.00,0.00,0.00,1005,376743,376743_1005_nan_493.5,376743_1005_836568_493.5,OPTBLUE_0,8138464136_OPTBLUE
1,TRANSACTN,8138352885,2,1180082360,2022-04-29,MXN,8138464136,2022-04-28,2022-04-28,133120,...,0.00,0.00,0.00,0.00,8858,379180,379180_8858_nan_1000.0,379180_8858_343305_1000.0,OPTBLUE_0,8138464136_OPTBLUE
2,TRANSACTN,8138352885,2,1180082360,2022-04-29,MXN,8138464227,2022-04-28,2022-04-28,133156,...,0.00,0.00,0.00,0.00,2009,376678,376678_2009_nan_20000.0,376678_2009_826421_20000.0,OPTBLUE_0,8138464227_OPTBLUE
3,TRANSACTN,8138352885,2,1100002672,2022-04-21,MXN,8138464193,2022-04-20,2022-04-20,111011,...,0.00,0.00,0.00,0.00,2005,376702,376702_2005_ONFEXJTBKRP6_3150.0,376702_2005_822633_3150.0,OPTBLUE_0,8138464193_OPTBLUE
4,TRANSACTN,8138352885,2,1090092150,2022-04-20,MXN,8138464136,2022-04-19,2022-04-19,110016,...,0.00,0.00,0.00,0.00,4807,379180,379180_4807_YESOMFJXXEXJ_370.0,379180_4807_645347_370.0,OPTBLUE_0,8138464136_OPTBLUE


In [64]:
opt.shape

(92793, 53)

In [65]:
opt.columns

Index(['1', '2', '3', '4', 'fecha_pago', '6', 'numero_establecimiento',
       'fecha_presentacion', 'fecha_procesamiento', '10', '11', '12',
       'order_id', '14', 'no_tarjeta', '16', '17', '18', 'fecha_transaccion',
       'hora_transaccion', 'numero_transaccion', 'codigo_aprobacion',
       'numero_terminal', '24', '25', 'referencia_adquiriente', '27', '28',
       '29', '30', '31', '32', 'rechazo_amex', '34', '35', 'cantidad_cuotas',
       'numero_cuota', '38', '39', '40', '41', 'Monto_bruto_presentacion',
       'Monto_transaccion', 'Monto_primera_cuota', 'Monto_cuota_siguiente',
       'Monto_servicio', 'Monto_aceleracion', 'last4_bank', 'bin_bank',
       'INDEX_TOTAL', 'INDEX_PARCIAL', 'index_comisiones',
       'index_comisiones_industria'],
      dtype='object')

### DATA FINAL

In [66]:
provision_optb_final = provision_optb.merge(opt[['order_id','fecha_transaccion','fecha_pago']]
                                        ,on='order_id',how='left')
#provision_amex_final.fillna('RETAINED',inplace=True)
provision_optb_final.rename(columns ={'Date_process_roc': 'date_process','roc_deposit_date': 'deposit_date'},inplace=True)
provision_optb_final.head()

,merchant_id,transaction_id,order_id,psp,affiliation_type,clave_autorizacion,date_transaction,amount,bin,last4,status_code,msi,clip_reader,report_id,report_date_local,status,CONCAT,fecha_transaccion,fecha_pago
0,88615f3b-6b3e-4e57-8604-b83fffc6da75,38312d42-3784-482b-81d9-551ebf9b7113,FGTWH5CLJH7Q,AMEX,OPTBLUE,001501876279486,2022-04-28,10.00,375150,6778,1,0.00,Clip Plus 2,CZ3BFZZNK,2022-04-29,PAID,FGTWH5CLJH7Q-10.0-6778,2022-04-28,2022-04-29
1,f1395f50-916d-446b-b211-7d255696f51a,e015225b-01a6-4d61-a4bd-c75e38eb1fb4,ZE7DKJZOLXQ7,AMEX,OPTBLUE,001501873842484,2022-04-28,744.00,376702,1006,1,0.00,Clip Pro,M7CH6L4KB,2022-04-29,PAID,ZE7DKJZOLXQ7-744.0-1006,2022-04-28,2022-04-29
2,cd160dcf-d9d9-4453-9190-c79e221780b4,ef1dd50a-bb1c-4f8d-a22d-a87c37c6c046,CHD2BOI5LFYT,AMEX,OPTBLUE,001501879142483,2022-04-28,588.50,371776,1006,1,0.00,Clip Total,6V28TYBXT,2022-04-29,PAID,CHD2BOI5LFYT-588.0-1006,2022-04-28,2022-04-29
3,c2e0bdf0-e3b1-48b2-affc-2bc1e5e035a6,13a92d7b-1edd-43c0-85ca-bdac489e209b,SDWEASRTEAJG,AMEX,OPTBLUE,001501876284486,2022-04-28,300.00,371773,1000,1,0.00,Clip Total,XS7JF3FZ2,2022-04-29,PAID,SDWEASRTEAJG-300.0-1000,2022-04-28,2022-04-29
4,4183a913-fbfe-4eeb-bde8-99da3a109af0,bd22a45c-0769-4467-a0a9-a47ef85b68c7,QQKDOWUFNMFY,AMEX,OPTBLUE,001501877262481,2022-04-28,"3,979.00",376679,4005,1,0.00,Clip Pro,92GQ46QH8,2022-04-29,PAID,QQKDOWUFNMFY-3979.0-4005,2022-04-28,2022-04-29


In [67]:
provision_optb_final['fecha_pago'] = pd.to_datetime(provision_optb_final['fecha_pago'])
provision_optb_final['fecha_transaccion'] = pd.to_datetime(provision_optb_final['fecha_transaccion'])

In [68]:
provision_optb_final.head()

,merchant_id,transaction_id,order_id,psp,affiliation_type,clave_autorizacion,date_transaction,amount,bin,last4,status_code,msi,clip_reader,report_id,report_date_local,status,CONCAT,fecha_transaccion,fecha_pago
0,88615f3b-6b3e-4e57-8604-b83fffc6da75,38312d42-3784-482b-81d9-551ebf9b7113,FGTWH5CLJH7Q,AMEX,OPTBLUE,001501876279486,2022-04-28,10.00,375150,6778,1,0.00,Clip Plus 2,CZ3BFZZNK,2022-04-29,PAID,FGTWH5CLJH7Q-10.0-6778,2022-04-28,2022-04-29
1,f1395f50-916d-446b-b211-7d255696f51a,e015225b-01a6-4d61-a4bd-c75e38eb1fb4,ZE7DKJZOLXQ7,AMEX,OPTBLUE,001501873842484,2022-04-28,744.00,376702,1006,1,0.00,Clip Pro,M7CH6L4KB,2022-04-29,PAID,ZE7DKJZOLXQ7-744.0-1006,2022-04-28,2022-04-29
2,cd160dcf-d9d9-4453-9190-c79e221780b4,ef1dd50a-bb1c-4f8d-a22d-a87c37c6c046,CHD2BOI5LFYT,AMEX,OPTBLUE,001501879142483,2022-04-28,588.50,371776,1006,1,0.00,Clip Total,6V28TYBXT,2022-04-29,PAID,CHD2BOI5LFYT-588.0-1006,2022-04-28,2022-04-29
3,c2e0bdf0-e3b1-48b2-affc-2bc1e5e035a6,13a92d7b-1edd-43c0-85ca-bdac489e209b,SDWEASRTEAJG,AMEX,OPTBLUE,001501876284486,2022-04-28,300.00,371773,1000,1,0.00,Clip Total,XS7JF3FZ2,2022-04-29,PAID,SDWEASRTEAJG-300.0-1000,2022-04-28,2022-04-29
4,4183a913-fbfe-4eeb-bde8-99da3a109af0,bd22a45c-0769-4467-a0a9-a47ef85b68c7,QQKDOWUFNMFY,AMEX,OPTBLUE,001501877262481,2022-04-28,"3,979.00",376679,4005,1,0.00,Clip Pro,92GQ46QH8,2022-04-29,PAID,QQKDOWUFNMFY-3979.0-4005,2022-04-28,2022-04-29


In [69]:
provision_optb_final.shape

(22896, 19)

In [70]:
optb_not_found = provision_optb_final[(provision_optb_final['fecha_pago'] == 'NO ENCONTRADO EN EPA')]
optb_not_found.head()

,merchant_id,transaction_id,order_id,psp,affiliation_type,clave_autorizacion,date_transaction,amount,bin,last4,status_code,msi,clip_reader,report_id,report_date_local,status,CONCAT,fecha_transaccion,fecha_pago


In [71]:
optb_not_found.shape

(0, 19)

### PIVOT 

#### REPORT DATE LOCAL

In [72]:
pivot_optb_report_date_local = pd.pivot_table(provision_optb_final,index=['date_transaction']
                                              ,columns=['report_date_local'],values=['amount']
                                              ,aggfunc=np.sum,margins=True).reset_index()
pivot_optb_report_date_local.fillna(0,inplace=True)
pivot_optb_report_date_local.head()

date_transaction     amount                              \
report_date_local                  2022-04-28    2022-04-29    2022-04-30   
0                       2022-04-28 127,616.40 17,125,679.17    108,544.80   
1                       2022-04-29       0.00    270,342.01 16,939,388.78   
2                       2022-04-30       0.00          0.00     87,874.26   
3                              All 127,616.40 17,396,021.18 17,135,807.84   

                                                          
report_date_local    2022-05-01 2022-05-02           All  
0                          0.00       0.00 17,361,840.37  
1                     81,649.30       0.00 17,291,380.09  
2                 15,495,139.83  63,522.42 15,646,536.51  
3                 15,576,789.13  63,522.42 50,299,756.97

In [73]:
pivot_optb_report_date_local.shape

(4, 7)

#### FECHA DEPOSITO

In [74]:
pivot_optb_fecha_deposito = pd.pivot_table(provision_optb_final,index=['date_transaction']
                                              ,columns=['fecha_pago'],values=['amount']
                                              ,aggfunc=np.sum,margins=True).reset_index()
pivot_optb_fecha_deposito.fillna(0,inplace=True)
pivot_optb_fecha_deposito.head()

date_transaction              amount                      \
fecha_pago                  2022-04-29 00:00:00 2022-05-02 00:00:00   
0                2022-04-28       15,326,326.52        2,035,513.85   
1                2022-04-29                0.00       17,291,380.09   
2                2022-04-30                0.00       15,646,536.51   
3                       All       15,326,326.52       34,973,430.45   

                          
fecha_pago           All  
0          17,361,840.37  
1          17,291,380.09  
2          15,646,536.51  
3          50,299,756.97

In [75]:
pivot_optb_fecha_deposito.shape

(4, 4)

#### READER

In [76]:
pivot_reader_optblue = pd.pivot_table(provision_optb_final,index=['fecha_pago','clip_reader']
                                              ,columns=['report_date_local'],values=['amount']
                                              ,aggfunc=np.sum,margins=True).reset_index()
pivot_reader_optblue.fillna(0,inplace=True)
pivot_reader_optblue.head()

fecha_pago   clip_reader     amount               \
report_date_local                                    2022-04-28   2022-04-29   
0                  2022-04-29 00:00:00  Clip Classic   4,503.60   402,543.11   
1                  2022-04-29 00:00:00     Clip Plus  32,145.33 2,854,875.25   
2                  2022-04-29 00:00:00   Clip Plus 2  74,781.00 5,005,265.79   
3                  2022-04-29 00:00:00      Clip Pro  11,490.93 3,175,916.00   
4                  2022-04-29 00:00:00    Clip Total   4,695.54 3,658,787.17   

                                                                 
report_date_local 2022-04-30 2022-05-01 2022-05-02          All  
0                 101,322.80       0.00       0.00   508,369.51  
1                       0.00       0.00       0.00 2,887,020.58  
2                       0.00       0.00       0.00 5,080,046.79  
3                       0.00       0.00       0.00 3,187,406.93  
4                       0.00       0.00       0.00 3,663,482.71

In [77]:
pivot_reader_optblue.shape

(11, 8)

# PROVISIONES ECOMMERCE

In [78]:
provision_ecommerce = all_provisiones[(all_provisiones['psp'] == 'PAYWORKS-BANORTE')]
provision_ecommerce.rename(columns ={'order_id':'order_id_xpay'}, inplace=True)
provision_ecommerce['CONCAT'] = provision_ecommerce['order_id_xpay'].astype(str).replace(' ', '',regex=True)+'-'+provision_ecommerce['amount'].apply(np.floor).astype(str).replace(' ', '',regex=True)+'-'+provision_ecommerce['last4'].astype(str).replace(' ', '',regex=True)
provision_ecommerce.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_13792\1196054225.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  provision_ecommerce.rename(columns ={'order_id':'order_id_xpay'}, inplace=True)
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_13792\1196054225.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  provision_ecommerce['CONCAT'] = provision_ecommerce['order_id_xpay'].astype(str).replace(' ', '',regex=True)+'-'+provision_ecommerce['amount'].apply(np.floor).astype(str).replace(' ', '',regex=True)+'-'+provision_ecommerce['last4'].astype(

,merchant_id,transaction_id,order_id_xpay,psp,affiliation_type,clave_autorizacion,date_transaction,amount,bin,last4,status_code,msi,clip_reader,report_id,report_date_local,status,CONCAT
164,b10b31c0-555e-4cd3-a15b-de27edfd681a,c85150a2-c49e-481e-8c21-e8b403372eba,294338921563,PAYWORKS-BANORTE,ECOMMERCE,022104,2022-04-28,"5,000.00",547046,5885,1,0.00,Clip Classic,HCV2769QQ,2022-04-29,PAID,294338921563-5000.0-5885
215,c90d40c5-1602-465d-811b-7824f0d0beec,91b5c526-66a8-4340-b498-e72bd2558192,497405494488,PAYWORKS-BANORTE,ECOMMERCE,512542,2022-04-28,"2,100.00",526777,5482,1,0.00,Clip Classic,SZSRH6872,2022-04-29,PAID,497405494488-2100.0-5482
269,a7a20b54-f544-45e3-87df-60a3f5a62297,2ddcee0f-3a61-4cf4-82de-42194ed9480a,798726207755,PAYWORKS-BANORTE,ECOMMERCE,620561,2022-04-28,"2,500.00",519203,8587,1,0.00,Clip Classic,DJM8RNRK3,2022-04-29,PAID,798726207755-2500.0-8587
589,afab7f04-e18c-4425-9ea0-fb7b9358ab4b,253965e9-e71b-4762-8c91-41056201c983,595239016268,PAYWORKS-BANORTE,ECOMMERCE,198398,2022-04-28,200.00,529091,8442,1,0.00,Clip Classic,DY2QG3J9M,2022-04-29,PAID,595239016268-200.0-8442
721,c722b97d-8a8c-437e-9ae5-a1c72b15c772,73adec83-62c5-43fe-bfa1-2f097f6e69d4,251959117477,PAYWORKS-BANORTE,ECOMMERCE,440909,2022-04-28,"3,257.00",458909,2786,1,0.00,Clip Classic,4BWWYT79K,2022-04-29,PAID,251959117477-3257.0-2786


In [79]:
provision_ecommerce.shape

(2604, 17)

In [80]:
data_status_ecom_1 = provision_ecommerce.merge(sftp[['order_id_xpay','order_id', 'transaction_date', 'deposit_date',
                                                    'INDEX_TOTAL']],on='order_id_xpay',how='left')
data_status_ecom_1['transaction_date'].fillna('NO ENCONTRADO EN SFTP',inplace=True)
data_status_ecom_1['deposit_date'].fillna('NO ENCONTRADO EN SFTP',inplace=True)
#data_status_ecom_1.drop_duplicates(subset ="CONCAT", keep = 'last', inplace = True) 
data_status_ecom_1.head()

,merchant_id,transaction_id,order_id_xpay,psp,affiliation_type,clave_autorizacion,date_transaction,amount,bin,last4,...,msi,clip_reader,report_id,report_date_local,status,CONCAT,order_id,transaction_date,deposit_date,INDEX_TOTAL
0,b10b31c0-555e-4cd3-a15b-de27edfd681a,c85150a2-c49e-481e-8c21-e8b403372eba,294338921563,PAYWORKS-BANORTE,ECOMMERCE,022104,2022-04-28,"5,000.00",547046,5885,...,0.00,Clip Classic,HCV2769QQ,2022-04-29,PAID,294338921563-5000.0-5885,825741789662,2022-04-28 00:00:00,2022-04-29 00:00:00,547046_7085_825741789662_5000.0
1,c90d40c5-1602-465d-811b-7824f0d0beec,91b5c526-66a8-4340-b498-e72bd2558192,497405494488,PAYWORKS-BANORTE,ECOMMERCE,512542,2022-04-28,"2,100.00",526777,5482,...,0.00,Clip Classic,SZSRH6872,2022-04-29,PAID,497405494488-2100.0-5482,855845006915,2022-04-28 00:00:00,2022-04-29 00:00:00,526777_4182_855845006915_2100.0
2,a7a20b54-f544-45e3-87df-60a3f5a62297,2ddcee0f-3a61-4cf4-82de-42194ed9480a,798726207755,PAYWORKS-BANORTE,ECOMMERCE,620561,2022-04-28,"2,500.00",519203,8587,...,0.00,Clip Classic,DJM8RNRK3,2022-04-29,PAID,798726207755-2500.0-8587,926173427822,2022-04-28 00:00:00,2022-04-29 00:00:00,519203_0087_926173427822_2500.0
3,afab7f04-e18c-4425-9ea0-fb7b9358ab4b,253965e9-e71b-4762-8c91-41056201c983,595239016268,PAYWORKS-BANORTE,ECOMMERCE,198398,2022-04-28,200.00,529091,8442,...,0.00,Clip Classic,DY2QG3J9M,2022-04-29,PAID,595239016268-200.0-8442,926173428134,2022-04-28 00:00:00,2022-04-29 00:00:00,529091_9142_926173428134_200.0
4,c722b97d-8a8c-437e-9ae5-a1c72b15c772,73adec83-62c5-43fe-bfa1-2f097f6e69d4,251959117477,PAYWORKS-BANORTE,ECOMMERCE,440909,2022-04-28,"3,257.00",458909,2786,...,0.00,Clip Classic,4BWWYT79K,2022-04-29,PAID,251959117477-3257.0-2786,855845007451,2022-04-28 00:00:00,2022-04-29 00:00:00,458909_0186_855845007451_3257.0


In [81]:
data_status_ecom_1.shape

(2604, 21)

In [82]:
duplicates_status_ecom_1 = data_status_ecom_1[data_status_ecom_1.duplicated(['INDEX_TOTAL'] , keep=False)]
duplicates_status_ecom_1.head()

,merchant_id,transaction_id,order_id_xpay,psp,affiliation_type,clave_autorizacion,date_transaction,amount,bin,last4,...,msi,clip_reader,report_id,report_date_local,status,CONCAT,order_id,transaction_date,deposit_date,INDEX_TOTAL


In [83]:
duplicates_status_ecom_1.shape

(0, 21)

In [84]:
results_status_ecom_1 = data_status_ecom_1[(data_status_ecom_1['transaction_date'] != 'NO ENCONTRADO EN SFTP')]
results_status_ecom_1.head()

,merchant_id,transaction_id,order_id_xpay,psp,affiliation_type,clave_autorizacion,date_transaction,amount,bin,last4,...,msi,clip_reader,report_id,report_date_local,status,CONCAT,order_id,transaction_date,deposit_date,INDEX_TOTAL
0,b10b31c0-555e-4cd3-a15b-de27edfd681a,c85150a2-c49e-481e-8c21-e8b403372eba,294338921563,PAYWORKS-BANORTE,ECOMMERCE,022104,2022-04-28,"5,000.00",547046,5885,...,0.00,Clip Classic,HCV2769QQ,2022-04-29,PAID,294338921563-5000.0-5885,825741789662,2022-04-28 00:00:00,2022-04-29 00:00:00,547046_7085_825741789662_5000.0
1,c90d40c5-1602-465d-811b-7824f0d0beec,91b5c526-66a8-4340-b498-e72bd2558192,497405494488,PAYWORKS-BANORTE,ECOMMERCE,512542,2022-04-28,"2,100.00",526777,5482,...,0.00,Clip Classic,SZSRH6872,2022-04-29,PAID,497405494488-2100.0-5482,855845006915,2022-04-28 00:00:00,2022-04-29 00:00:00,526777_4182_855845006915_2100.0
2,a7a20b54-f544-45e3-87df-60a3f5a62297,2ddcee0f-3a61-4cf4-82de-42194ed9480a,798726207755,PAYWORKS-BANORTE,ECOMMERCE,620561,2022-04-28,"2,500.00",519203,8587,...,0.00,Clip Classic,DJM8RNRK3,2022-04-29,PAID,798726207755-2500.0-8587,926173427822,2022-04-28 00:00:00,2022-04-29 00:00:00,519203_0087_926173427822_2500.0
3,afab7f04-e18c-4425-9ea0-fb7b9358ab4b,253965e9-e71b-4762-8c91-41056201c983,595239016268,PAYWORKS-BANORTE,ECOMMERCE,198398,2022-04-28,200.00,529091,8442,...,0.00,Clip Classic,DY2QG3J9M,2022-04-29,PAID,595239016268-200.0-8442,926173428134,2022-04-28 00:00:00,2022-04-29 00:00:00,529091_9142_926173428134_200.0
4,c722b97d-8a8c-437e-9ae5-a1c72b15c772,73adec83-62c5-43fe-bfa1-2f097f6e69d4,251959117477,PAYWORKS-BANORTE,ECOMMERCE,440909,2022-04-28,"3,257.00",458909,2786,...,0.00,Clip Classic,4BWWYT79K,2022-04-29,PAID,251959117477-3257.0-2786,855845007451,2022-04-28 00:00:00,2022-04-29 00:00:00,458909_0186_855845007451_3257.0


In [85]:
results_status_ecom_1.shape

(2603, 21)

In [86]:
NA_status_ecom_1 = data_status_ecom_1[(data_status_ecom_1['transaction_date'] == 'NO ENCONTRADO EN SFTP')]
NA_status_ecom_1.head()

,merchant_id,transaction_id,order_id_xpay,psp,affiliation_type,clave_autorizacion,date_transaction,amount,bin,last4,...,msi,clip_reader,report_id,report_date_local,status,CONCAT,order_id,transaction_date,deposit_date,INDEX_TOTAL
1466,e38a2887-01f1-403f-a0a2-0ea5a01ac0ac,23da8f99-0908-4caa-93fc-0e537432d558,475063307787,PAYWORKS-BANORTE,ECOMMERCE,052094,2022-04-29,"2,088.00",553253,1652,...,0.00,Clip Classic,PHSRRL7Y3,2022-04-30,PAID,475063307787-2088.0-1652,NaN,NO ENCONTRADO EN SFTP,NO ENCONTRADO EN SFTP,NaN


In [87]:
NA_status_ecom_1.shape

(1, 21)

### DATA FINAL

In [88]:
provision_xpay_final = pd.concat([results_status_ecom_1,NA_status_ecom_1])
provision_xpay_final['status'].fillna('RETAINED',inplace=True)
provision_xpay_final['report_date_local'].fillna('NO ENCONTRADO',inplace=True)
provision_xpay_final.head()

,merchant_id,transaction_id,order_id_xpay,psp,affiliation_type,clave_autorizacion,date_transaction,amount,bin,last4,...,msi,clip_reader,report_id,report_date_local,status,CONCAT,order_id,transaction_date,deposit_date,INDEX_TOTAL
0,b10b31c0-555e-4cd3-a15b-de27edfd681a,c85150a2-c49e-481e-8c21-e8b403372eba,294338921563,PAYWORKS-BANORTE,ECOMMERCE,022104,2022-04-28,"5,000.00",547046,5885,...,0.00,Clip Classic,HCV2769QQ,2022-04-29,PAID,294338921563-5000.0-5885,825741789662,2022-04-28 00:00:00,2022-04-29 00:00:00,547046_7085_825741789662_5000.0
1,c90d40c5-1602-465d-811b-7824f0d0beec,91b5c526-66a8-4340-b498-e72bd2558192,497405494488,PAYWORKS-BANORTE,ECOMMERCE,512542,2022-04-28,"2,100.00",526777,5482,...,0.00,Clip Classic,SZSRH6872,2022-04-29,PAID,497405494488-2100.0-5482,855845006915,2022-04-28 00:00:00,2022-04-29 00:00:00,526777_4182_855845006915_2100.0
2,a7a20b54-f544-45e3-87df-60a3f5a62297,2ddcee0f-3a61-4cf4-82de-42194ed9480a,798726207755,PAYWORKS-BANORTE,ECOMMERCE,620561,2022-04-28,"2,500.00",519203,8587,...,0.00,Clip Classic,DJM8RNRK3,2022-04-29,PAID,798726207755-2500.0-8587,926173427822,2022-04-28 00:00:00,2022-04-29 00:00:00,519203_0087_926173427822_2500.0
3,afab7f04-e18c-4425-9ea0-fb7b9358ab4b,253965e9-e71b-4762-8c91-41056201c983,595239016268,PAYWORKS-BANORTE,ECOMMERCE,198398,2022-04-28,200.00,529091,8442,...,0.00,Clip Classic,DY2QG3J9M,2022-04-29,PAID,595239016268-200.0-8442,926173428134,2022-04-28 00:00:00,2022-04-29 00:00:00,529091_9142_926173428134_200.0
4,c722b97d-8a8c-437e-9ae5-a1c72b15c772,73adec83-62c5-43fe-bfa1-2f097f6e69d4,251959117477,PAYWORKS-BANORTE,ECOMMERCE,440909,2022-04-28,"3,257.00",458909,2786,...,0.00,Clip Classic,4BWWYT79K,2022-04-29,PAID,251959117477-3257.0-2786,855845007451,2022-04-28 00:00:00,2022-04-29 00:00:00,458909_0186_855845007451_3257.0


In [89]:
provision_xpay_final.shape

(2604, 21)

### PIVOT

#### REPORT_DATE_LOCAL

In [90]:
pivot_xpay_report_date_local = pd.pivot_table(provision_xpay_final,index=['date_transaction']
                                              ,columns=['report_date_local'],values=['amount']
                                              ,aggfunc=np.sum,margins=True).reset_index()
pivot_xpay_report_date_local.fillna(0,inplace=True)
pivot_xpay_report_date_local.head()

date_transaction       amount                          \
report_date_local                    2022-04-29   2022-04-30 2022-05-01   
0                       2022-04-28 1,180,725.76         0.00       0.00   
1                       2022-04-29         0.00 1,370,708.81       0.00   
2                       2022-04-30         0.00         0.00 904,665.26   
3                              All 1,180,725.76 1,370,708.81 904,665.26   

                                
report_date_local          All  
0                 1,180,725.76  
1                 1,370,708.81  
2                   904,665.26  
3                 3,456,099.83

In [91]:
pivot_xpay_report_date_local.shape

(4, 5)

#### FECHA_DEPOSITO

In [92]:
pivot_xpay_fecha_deposito = pd.pivot_table(provision_xpay_final,index=['date_transaction']
                                              ,columns=['deposit_date'],values=['amount']
                                              ,aggfunc=np.sum,margins=True).reset_index()
pivot_xpay_fecha_deposito.fillna(0,inplace=True)
pivot_xpay_fecha_deposito.head()

date_transaction              amount                      \
deposit_date                  2022-04-29 00:00:00 2022-05-02 00:00:00   
0                  2022-04-28        1,138,954.96           41,770.80   
1                  2022-04-29                0.00        1,368,620.81   
2                  2022-04-30                0.00          904,665.26   
3                         All        1,138,954.96        2,315,056.87   

                                                 
deposit_date NO ENCONTRADO EN SFTP          All  
0                             0.00 1,180,725.76  
1                         2,088.00 1,370,708.81  
2                             0.00   904,665.26  
3                         2,088.00 3,456,099.83

In [93]:
pivot_xpay_fecha_deposito.shape

(4, 5)

#### READER

In [94]:
pivot_reader_xpay = pd.pivot_table(provision_xpay_final,index=['deposit_date','clip_reader']
                                              ,columns=['report_date_local'],values=['amount']
                                              ,aggfunc=np.sum,margins=True).reset_index(1)
pivot_reader_xpay.fillna(0,inplace=True)
pivot_reader_xpay.head()

clip_reader       amount                          \
report_date_local                     2022-04-29   2022-04-30 2022-05-01   
deposit_date                                                               
2022-04-29 00:00:00    Clip Classic 1,138,954.96         0.00       0.00   
2022-05-02 00:00:00    Clip Classic    41,770.80 1,368,620.81 904,665.26   
NO ENCONTRADO EN SFTP  Clip Classic         0.00     2,088.00       0.00   
All                                 1,180,725.76 1,370,708.81 904,665.26   

                                    
report_date_local              All  
deposit_date                        
2022-04-29 00:00:00   1,138,954.96  
2022-05-02 00:00:00   2,315,056.87  
NO ENCONTRADO EN SFTP     2,088.00  
All                   3,456,099.83

In [95]:
pivot_reader_xpay.shape

(4, 5)

In [96]:
# book = load_workbook('PROVISIONES SEPTIEMBRE 2020.xlsx')
# writer = pd.ExcelWriter('PROVISIONES SEPTIEMBRE 2020.xlsx', engine='openpyxl', options={'encoding':'utf-8'})
# writer.book = book
# provision_ecommerce_total.to_excel(writer, sheet_name='SEPTIEMBRE XPAY', index=False)
# writer.save()
# writer.close()

# PROVISIONES MIT-3DS

In [97]:
provision_mit = all_provisiones[(all_provisiones['psp'] == 'MIT-3DS')]
provision_mit['CONCAT'] = provision_mit['order_id'].astype(str).replace(' ', '',regex=True)+'-'+provision_mit['amount'].apply(np.floor).astype(str).replace(' ', '',regex=True)+'-'+provision_mit['last4'].astype(str).replace(' ', '',regex=True)
provision_mit.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_13792\3654006801.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  provision_mit['CONCAT'] = provision_mit['order_id'].astype(str).replace(' ', '',regex=True)+'-'+provision_mit['amount'].apply(np.floor).astype(str).replace(' ', '',regex=True)+'-'+provision_mit['last4'].astype(str).replace(' ', '',regex=True)


,merchant_id,transaction_id,order_id,psp,affiliation_type,clave_autorizacion,date_transaction,amount,bin,last4,status_code,msi,clip_reader,report_id,report_date_local,status,CONCAT
1818,c25b1154-f99e-439a-86cc-364d61dfbf3d,42eefb5a-eeba-4190-a7c6-424a702270af,000616978404,MIT-3DS,ECOMMERCE,846706,2022-04-28,500.00,526777,6105,1,0.00,Clip Classic,GSZ7X86KB,2022-04-29,PAID,000616978404-500.0-6105
2922,19a2c879-e97b-4a23-8fa2-52f759142ff6,1cd306be-c65c-473d-a539-5eb2a38eee9a,000616986373,MIT-3DS,ECOMMERCE,122716,2022-04-28,"3,372.00",526777,9133,1,0.00,Clip Classic,DMGGYJMM2,2022-04-29,PAID,000616986373-3372.0-9133
4317,0424f322-a940-4502-8407-621a670e1dbb,3a7e7132-d81f-42e3-a1d2-b8aac570ff1a,000616991922,MIT-3DS,ECOMMERCE,005941,2022-04-28,"4,500.00",421316,7242,1,0.00,Clip Classic,KLLT737HV,2022-04-29,PAID,000616991922-4500.0-7242
5547,48bcdb5d-d46a-40e4-a1e4-68c5ddb9d138,1367bb5b-ef68-4577-ad5e-4aae7e63dfcc,000617001764,MIT-3DS,ECOMMERCE,010708,2022-04-28,"3,600.26",546759,9426,1,0.00,Clip Classic,DXB47XDY8,2022-04-29,PAID,000617001764-3600.0-9426
14800,328e58dc-8306-46b1-8959-1f100361ed7c,a185d1c9-8f4f-42af-a1dd-f5693707f799,000617056559,MIT-3DS,ECOMMERCE,296041,2022-04-28,"2,640.00",474175,1154,1,3.00,Clip Classic,PWNRBBQV8,2022-04-29,PAID,000617056559-2640.0-1154


In [98]:
provision_mit.shape

(387, 17)

In [99]:
data_status_mit = provision_mit.merge(sftp[['order_id', 'transaction_date', 'deposit_date','INDEX_TOTAL']],on='order_id',how='left')
data_status_mit['transaction_date'].fillna('NO ENCONTRADO EN SFTP',inplace=True)
data_status_mit['deposit_date'].fillna('NO ENCONTRADO EN SFTP',inplace=True)
data_status_mit.drop_duplicates(subset ="CONCAT", keep = 'last', inplace = True) 
data_status_mit.head()

,merchant_id,transaction_id,order_id,psp,affiliation_type,clave_autorizacion,date_transaction,amount,bin,last4,status_code,msi,clip_reader,report_id,report_date_local,status,CONCAT,transaction_date,deposit_date,INDEX_TOTAL
0,c25b1154-f99e-439a-86cc-364d61dfbf3d,42eefb5a-eeba-4190-a7c6-424a702270af,000616978404,MIT-3DS,ECOMMERCE,846706,2022-04-28,500.00,526777,6105,1,0.00,Clip Classic,GSZ7X86KB,2022-04-29,PAID,000616978404-500.0-6105,2022-04-28,2022-04-29,526777_5005_000616978404_500.0
1,19a2c879-e97b-4a23-8fa2-52f759142ff6,1cd306be-c65c-473d-a539-5eb2a38eee9a,000616986373,MIT-3DS,ECOMMERCE,122716,2022-04-28,"3,372.00",526777,9133,1,0.00,Clip Classic,DMGGYJMM2,2022-04-29,PAID,000616986373-3372.0-9133,2022-04-28,2022-04-29,526777_0033_000616986373_3372.0
2,0424f322-a940-4502-8407-621a670e1dbb,3a7e7132-d81f-42e3-a1d2-b8aac570ff1a,000616991922,MIT-3DS,ECOMMERCE,005941,2022-04-28,"4,500.00",421316,7242,1,0.00,Clip Classic,KLLT737HV,2022-04-29,PAID,000616991922-4500.0-7242,2022-04-28,2022-04-29,421316_6142_000616991922_4500.0
3,48bcdb5d-d46a-40e4-a1e4-68c5ddb9d138,1367bb5b-ef68-4577-ad5e-4aae7e63dfcc,000617001764,MIT-3DS,ECOMMERCE,010708,2022-04-28,"3,600.26",546759,9426,1,0.00,Clip Classic,DXB47XDY8,2022-04-29,PAID,000617001764-3600.0-9426,2022-04-28,2022-04-29,546759_2026_000617001764_3600.26
4,328e58dc-8306-46b1-8959-1f100361ed7c,a185d1c9-8f4f-42af-a1dd-f5693707f799,000617056559,MIT-3DS,ECOMMERCE,296041,2022-04-28,"2,640.00",474175,1154,1,3.00,Clip Classic,PWNRBBQV8,2022-04-29,PAID,000617056559-2640.0-1154,2022-04-28,2022-04-29,474175_1154_000617056559_2640.0


In [100]:
data_status_mit.shape

(387, 20)

In [101]:
duplicates_status_mit = data_status_mit[data_status_mit.duplicated(['INDEX_TOTAL'] , keep=False)]
duplicates_status_mit.head()

,merchant_id,transaction_id,order_id,psp,affiliation_type,clave_autorizacion,date_transaction,amount,bin,last4,status_code,msi,clip_reader,report_id,report_date_local,status,CONCAT,transaction_date,deposit_date,INDEX_TOTAL


In [102]:
duplicates_status_mit.shape

(0, 20)

In [103]:
results_status_mit = data_status_mit[(data_status_mit['transaction_date'] != 'NO ENCONTRADO EN SFTP')]
results_status_mit.head()

,merchant_id,transaction_id,order_id,psp,affiliation_type,clave_autorizacion,date_transaction,amount,bin,last4,status_code,msi,clip_reader,report_id,report_date_local,status,CONCAT,transaction_date,deposit_date,INDEX_TOTAL
0,c25b1154-f99e-439a-86cc-364d61dfbf3d,42eefb5a-eeba-4190-a7c6-424a702270af,000616978404,MIT-3DS,ECOMMERCE,846706,2022-04-28,500.00,526777,6105,1,0.00,Clip Classic,GSZ7X86KB,2022-04-29,PAID,000616978404-500.0-6105,2022-04-28,2022-04-29,526777_5005_000616978404_500.0
1,19a2c879-e97b-4a23-8fa2-52f759142ff6,1cd306be-c65c-473d-a539-5eb2a38eee9a,000616986373,MIT-3DS,ECOMMERCE,122716,2022-04-28,"3,372.00",526777,9133,1,0.00,Clip Classic,DMGGYJMM2,2022-04-29,PAID,000616986373-3372.0-9133,2022-04-28,2022-04-29,526777_0033_000616986373_3372.0
2,0424f322-a940-4502-8407-621a670e1dbb,3a7e7132-d81f-42e3-a1d2-b8aac570ff1a,000616991922,MIT-3DS,ECOMMERCE,005941,2022-04-28,"4,500.00",421316,7242,1,0.00,Clip Classic,KLLT737HV,2022-04-29,PAID,000616991922-4500.0-7242,2022-04-28,2022-04-29,421316_6142_000616991922_4500.0
3,48bcdb5d-d46a-40e4-a1e4-68c5ddb9d138,1367bb5b-ef68-4577-ad5e-4aae7e63dfcc,000617001764,MIT-3DS,ECOMMERCE,010708,2022-04-28,"3,600.26",546759,9426,1,0.00,Clip Classic,DXB47XDY8,2022-04-29,PAID,000617001764-3600.0-9426,2022-04-28,2022-04-29,546759_2026_000617001764_3600.26
4,328e58dc-8306-46b1-8959-1f100361ed7c,a185d1c9-8f4f-42af-a1dd-f5693707f799,000617056559,MIT-3DS,ECOMMERCE,296041,2022-04-28,"2,640.00",474175,1154,1,3.00,Clip Classic,PWNRBBQV8,2022-04-29,PAID,000617056559-2640.0-1154,2022-04-28,2022-04-29,474175_1154_000617056559_2640.0


In [104]:
results_status_mit.shape

(387, 20)

In [105]:
NA_status_mit = data_status_mit[(data_status_mit['transaction_date'] == 'NO ENCONTRADO EN SFTP')]
NA_status_mit.head()

,merchant_id,transaction_id,order_id,psp,affiliation_type,clave_autorizacion,date_transaction,amount,bin,last4,status_code,msi,clip_reader,report_id,report_date_local,status,CONCAT,transaction_date,deposit_date,INDEX_TOTAL


In [106]:
NA_status_mit.shape

(0, 20)

### DATA FINAL

In [107]:
provision_mit_final = pd.concat([results_status_mit,NA_status_mit])
provision_mit_final['status'].fillna('RETAINED',inplace=True)
provision_mit_final['report_date_local'].fillna('NO ENCONTRADO',inplace=True)
provision_mit_final.head()

,merchant_id,transaction_id,order_id,psp,affiliation_type,clave_autorizacion,date_transaction,amount,bin,last4,status_code,msi,clip_reader,report_id,report_date_local,status,CONCAT,transaction_date,deposit_date,INDEX_TOTAL
0,c25b1154-f99e-439a-86cc-364d61dfbf3d,42eefb5a-eeba-4190-a7c6-424a702270af,000616978404,MIT-3DS,ECOMMERCE,846706,2022-04-28,500.00,526777,6105,1,0.00,Clip Classic,GSZ7X86KB,2022-04-29,PAID,000616978404-500.0-6105,2022-04-28,2022-04-29,526777_5005_000616978404_500.0
1,19a2c879-e97b-4a23-8fa2-52f759142ff6,1cd306be-c65c-473d-a539-5eb2a38eee9a,000616986373,MIT-3DS,ECOMMERCE,122716,2022-04-28,"3,372.00",526777,9133,1,0.00,Clip Classic,DMGGYJMM2,2022-04-29,PAID,000616986373-3372.0-9133,2022-04-28,2022-04-29,526777_0033_000616986373_3372.0
2,0424f322-a940-4502-8407-621a670e1dbb,3a7e7132-d81f-42e3-a1d2-b8aac570ff1a,000616991922,MIT-3DS,ECOMMERCE,005941,2022-04-28,"4,500.00",421316,7242,1,0.00,Clip Classic,KLLT737HV,2022-04-29,PAID,000616991922-4500.0-7242,2022-04-28,2022-04-29,421316_6142_000616991922_4500.0
3,48bcdb5d-d46a-40e4-a1e4-68c5ddb9d138,1367bb5b-ef68-4577-ad5e-4aae7e63dfcc,000617001764,MIT-3DS,ECOMMERCE,010708,2022-04-28,"3,600.26",546759,9426,1,0.00,Clip Classic,DXB47XDY8,2022-04-29,PAID,000617001764-3600.0-9426,2022-04-28,2022-04-29,546759_2026_000617001764_3600.26
4,328e58dc-8306-46b1-8959-1f100361ed7c,a185d1c9-8f4f-42af-a1dd-f5693707f799,000617056559,MIT-3DS,ECOMMERCE,296041,2022-04-28,"2,640.00",474175,1154,1,3.00,Clip Classic,PWNRBBQV8,2022-04-29,PAID,000617056559-2640.0-1154,2022-04-28,2022-04-29,474175_1154_000617056559_2640.0


In [108]:
provision_mit_final.shape

(387, 20)

### PIVOT 

#### REPORT_DATE_LOCAL

In [109]:
pivot_mit_report_date_local = pd.pivot_table(provision_mit_final,index=['date_transaction']
                                              ,columns=['report_date_local'],values=['amount']
                                              ,aggfunc=np.sum,margins=True).reset_index()
pivot_mit_report_date_local.fillna(0,inplace=True)
pivot_mit_report_date_local.head()

date_transaction     amount                        \
report_date_local                  2022-04-29 2022-04-30 2022-05-01   
0                       2022-04-28 487,815.97       0.00       0.00   
1                       2022-04-29       0.00 440,481.08       0.00   
2                       2022-04-30       0.00       0.00 253,392.32   
3                              All 487,815.97 440,481.08 253,392.32   

                                
report_date_local          All  
0                   487,815.97  
1                   440,481.08  
2                   253,392.32  
3                 1,181,689.37

In [110]:
pivot_mit_report_date_local.shape

(4, 5)

#### FECHA_DEPOSITO

In [111]:
pivot_mit_fecha_deposito = pd.pivot_table(provision_mit_final,index=['date_transaction']
                                              ,columns=['deposit_date'],values=['amount']
                                              ,aggfunc=np.sum,margins=True).reset_index()
pivot_mit_fecha_deposito.fillna(0,inplace=True)
pivot_mit_fecha_deposito.head()

date_transaction              amount                      \
deposit_date                  2022-04-29 00:00:00 2022-05-02 00:00:00   
0                  2022-04-28          468,567.77           19,248.20   
1                  2022-04-29                0.00          440,481.08   
2                  2022-04-30                0.00          253,392.32   
3                         All          468,567.77          713,121.60   

                           
deposit_date          All  
0              487,815.97  
1              440,481.08  
2              253,392.32  
3            1,181,689.37

In [112]:
pivot_mit_fecha_deposito.shape

(4, 4)

#### READER 

In [113]:
pivot_reader_mit = pd.pivot_table(provision_mit_final,index=['deposit_date','clip_reader']
                                              ,columns=['report_date_local'],values=['amount']
                                              ,aggfunc=np.sum,margins=True).reset_index(1)
pivot_reader_mit.fillna(0,inplace=True)
pivot_reader_mit.head()

clip_reader     amount                        \
report_date_local                 2022-04-29 2022-04-30 2022-05-01   
deposit_date                                                         
2022-04-29 00:00:00  Clip Classic 468,567.77       0.00       0.00   
2022-05-02 00:00:00  Clip Classic  19,248.20 440,481.08 253,392.32   
All                               487,815.97 440,481.08 253,392.32   

                                  
report_date_local            All  
deposit_date                      
2022-04-29 00:00:00   468,567.77  
2022-05-02 00:00:00   713,121.60  
All                 1,181,689.37

In [114]:
pivot_reader_mit.shape

(3, 5)

In [115]:
# book = load_workbook('PROVISIONES SEPTIEMBRE 2020.xlsx')
# writer = pd.ExcelWriter('PROVISIONES SEPTIEMBRE 2020.xlsx', engine='openpyxl', options={'encoding':'utf-8'})
# writer.book = book
# provision_mit_total.to_excel(writer, sheet_name='SEPTIEMBRE OPENPAY', index=False)
# writer.save()
# writer.close()

# PROVISIONES OPEN PAY (NO EJECUTAR EN CASO DE QUE EL PSP NO APAREZCA EN LA DB)

In [116]:
provision_openpay = all_provisiones[(all_provisiones['psp'] == 'OPENPAY')]
provision_openpay['CONCAT'] = provision_openpay['order_id'].astype(str).replace(' ', '',regex=True)+'-'+provision_openpay['amount'].apply(np.floor).astype(str).replace(' ', '',regex=True)+'-'+provision_openpay['last4'].astype(str).replace(' ', '',regex=True)
provision_openpay.head()

,merchant_id,transaction_id,order_id,psp,affiliation_type,clave_autorizacion,date_transaction,amount,bin,last4,status_code,msi,clip_reader,report_id,report_date_local,status,CONCAT


In [117]:
provision_openpay.shape

(0, 17)

### DATA FINAL

In [ ]:
provision_openpay_final = pd.concat([provision_openpay])
provision_openpay_final.fillna('RETAINED',inplace=True)
provision_openpay_final.head()

In [ ]:
provision_openpay_final.shape

### PIVOT 

#### REPORT_DATE_LOCAL

In [ ]:
pivot_openpay_report_date_local = pd.pivot_table(provision_openpay_final,index=['date_transaction']
                                              ,columns=['report_date_local'],values=['amount']
                                              ,aggfunc=np.sum,margins=True).reset_index()
pivot_openpay_report_date_local.fillna(0,inplace=True)
pivot_openpay_report_date_local.head()

In [ ]:
pivot_openpay_report_date_local.shape

#### READER

In [ ]:
pivot_reader_openpay = pd.pivot_table(provision_openpay_final,index=['clip_reader']
                                              ,columns=['report_date_local'],values=['amount']
                                              ,aggfunc=np.sum,margins=True).reset_index()
pivot_reader_openpay.fillna(0,inplace=True)
pivot_reader_openpay.head()

In [ ]:
pivot_reader_openpay.shape

In [ ]:
# book = load_workbook('PROVISIONES SEPTIEMBRE 2020.xlsx')
# writer = pd.ExcelWriter('PROVISIONES SEPTIEMBRE 2020.xlsx', engine='openpyxl', options={'encoding':'utf-8'})
# writer.book = book
# provision_openpay_final.to_excel(writer, sheet_name='SEPTIEMBRE OPENPAY', index=False)
# writer.save()
# writer.close()

# EXCEL FINAL 

In [118]:
# Creating Excel Writer Object from Pandas  
writer = pd.ExcelWriter('PROVISIÓN ME ABRIL 2022.xlsx',engine='xlsxwriter',engine_kwargs={'options': {'encoding':'utf-8'}})  
workbook=writer.book

#RESUMEN
worksheet=workbook.add_worksheet('RESUMEN FINAL')
writer.sheets['RESUMEN FINAL'] = worksheet
pivot_reader_bnte.to_excel(writer,sheet_name='RESUMEN FINAL',startrow=0 , startcol=0)   
pivot_reader_amex.to_excel(writer,sheet_name='RESUMEN FINAL',startrow=21, startcol=0)
pivot_reader_optblue.to_excel(writer,sheet_name='RESUMEN FINAL',startrow=41, startcol=0)
pivot_reader_xpay.to_excel(writer,sheet_name='RESUMEN FINAL',startrow=61, startcol=0)
pivot_reader_mit.to_excel(writer,sheet_name='RESUMEN FINAL',startrow=81, startcol=0)

# BNTE
worksheet=workbook.add_worksheet('BNTE')
writer.sheets['BNTE'] = worksheet
group_1.to_excel(writer,sheet_name='BNTE_1',startrow=0 , startcol=0, index=False)
group_2.to_excel(writer,sheet_name='BNTE_2',startrow=0 , startcol=0, index=False)
group_3.to_excel(writer,sheet_name='BNTE_3',startrow=0 , startcol=0, index=False)
pivot_bnte_report_date_local.to_excel(writer,sheet_name='BNTE',startrow=1, startcol=21)
pivot_bnte_fecha_deposito.to_excel(writer,sheet_name='BNTE',startrow=12, startcol=21)

# AMEX
worksheet=workbook.add_worksheet('AMEX')
writer.sheets['AMEX'] = worksheet
results_amex_final.to_excel(writer,sheet_name='AMEX',startrow=0 , startcol=0, index=False)  
pivot_amex_report_date_local.to_excel(writer,sheet_name='AMEX',startrow=1, startcol=21)
pivot_amex_fecha_deposito.to_excel(writer,sheet_name='AMEX',startrow=12, startcol=21)

#OPTBLUE
worksheet=workbook.add_worksheet('OPTBLUE')
writer.sheets['OPTBLUE'] = worksheet
provision_optb_final.to_excel(writer,sheet_name='OPTBLUE',startrow=0 , startcol=0, index=False)  
pivot_optb_report_date_local.to_excel(writer,sheet_name='OPTBLUE',startrow=1, startcol=21)
pivot_optb_fecha_deposito.to_excel(writer,sheet_name='OPTBLUE',startrow=12, startcol=21)

# XPAY
worksheet=workbook.add_worksheet('XPAY')
writer.sheets['XPAY'] = worksheet
provision_xpay_final.to_excel(writer,sheet_name='XPAY',startrow=0 , startcol=0, index=False)  
pivot_xpay_report_date_local.to_excel(writer,sheet_name='XPAY',startrow=1, startcol=22)
pivot_xpay_fecha_deposito.to_excel(writer,sheet_name='XPAY',startrow=12, startcol=22)

# MIT
worksheet=workbook.add_worksheet('MIT')
writer.sheets['MIT'] = worksheet
provision_mit_final.to_excel(writer,sheet_name='MIT',startrow=0 , startcol=0, index=False)  
pivot_mit_report_date_local.to_excel(writer,sheet_name='MIT',startrow=1, startcol=22)
pivot_mit_fecha_deposito.to_excel(writer,sheet_name='MIT',startrow=12, startcol=22)

# OPENPAY
# worksheet=workbook.add_worksheet('OPENPAY')
# writer.sheets['OPENPAY'] = worksheet
# provision_openpay_final.to_excel(writer,sheet_name='OPENPAY',startrow=0 , startcol=0, index=False)  
# pivot_openpay_report_date_local.to_excel(writer,sheet_name='OPENPAY',startrow=1, startcol=21)

writer.close()